In [1]:
import pandas as pd
import numpy as np
import openpyxl
from lower_case_dict import d_dict
from lower_case_dict import D_titulo_grado_a_rama
from lower_case_dict import D_titulo_grado_a_ambito
from lower_case_dict import D_ambito_a_rama
from lower_case_dict import D_ocupaciones
from lower_case_dict import D_cnae_a_num
from upper_case_dict_Grado14 import D_dict 
pd.options.display.max_columns=None
pd.options.display.max_colwidth=300

# Corregir
Los **diccionarios de las variables originales** en la encuesta y los definidos para las **variables nuevas** (creadas inicialmente para el dataset de graduados 2019) se importan, con algunas excepciones, del **módulo lower_case_dict**:  

- **D_dict**; diccionarios de traducción de las variables categóricas. Construidos de acuerdo con los códigos facilitados en el archivo de diseño de la encuesta 'dr_EILU_GRAD_2019.xlsx' y con los definidos para las variables nuevas creadas en la inspeccion de la encuesta de graduados 2019. Para cada variable: D_dict[variable][código_categórico]=interpretación_código. 
- **D_describe**; diccionario de interpretación de las variables originales y nuevas: D_describe[variable]=interpretación_variable.  

Las **excepciones** son algunos diccionnarios que no son iguales para las encuestas a graduados y titulados de máster 2019. Estos se importan a través de la función diccionario definida en el **módulo upper_case_dict_Master19**, una adaptación del módulo módulo upper_case_dict ajustada a la encuesta de titulados de máster 2019. Se invocan y aplican caso a caso cuando procede. 

En el proceso de inspección se redefinen algunas variables originales y se definen otras nuevas. Estas se añaden a los diccionarios D_dict y D_describe para ser recuperados con el módulo lower_case_dict.  

También se importa del módulo lower_case_dict el diccionario D_ambito_to_rama que asocia los ámbitos de estudios a su rama de conocimiento. 

Se **redefinen** las **funciones de traducción**: 

In [2]:
def traduce_upper(col):
    """ Muestra la leyenda de las variables originales de la encuesta (denominadas con mayúsculas) de acuerdo 
    con su diccionario específico. Útil para comprobar las correspondencias con los diccionarios del resto de 
    encuestas. No devuelve una variable traducida
    """
    global df, D_dict
    # Resuelve confictos con col. mixtas (strings e integers) y con Nan
    df['aux_traduce']=df[col].astype('O').replace(D_dict[col])  
    print (df[[col,'aux_traduce']].groupby(col).agg(['unique'])) # Salida de comprobación
    return 

In [3]:
def traduce_lower(col):
    """ Traduce las variables transformadas (denominadas con minúsculas) de acuerdo con los diccionarios
    generales para ellas y devuelve la variable traducida. 
    """
    global df, d_dict
    # Resuelve confictos con col. mixtas (strings e integers) y con Nan
    df['aux_traduce']=df[col].astype('O').replace(d_dict[col])  
    print (df[[col,'aux_traduce']].groupby(col).agg(['unique'])) # Salida de comprobación
    return df['aux_traduce']

# Recuperación del dataset
Dataset 'Encuesta_insercion_laboral_titulados_universitarios_2019' en formato csv.   

Enlace a fuente (INE): https://www.ine.es/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736176991&menu=resultados&idp=1254735573113#!tabs-1254736195339

In [4]:
ruta_dataset='data/EILU_2014.csv'

In [5]:
df_raw=pd.read_csv(ruta_dataset,sep='\t')

/home/bea/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (32,111,112,113,114,115,116,117,118,119,121,122,123) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
df=df_raw.copy()

In [7]:
df.shape

(30379, 180)

In [8]:
for n,col in enumerate(df.columns):
    print(n,col,df[col].dtype)

0 IDENT int64
1 SEXO int64
2 EDAD int64
3 NACIO int64
4 TITU int64
5 RAMA int64
6 AMBITO int64
7 T_UNIV int64
8 DISCA int64
9 PROXY int64
10 EST_B1 object
11 EST_B21 object
12 EST_B22 object
13 EST_B23 object
14 EST_B24 object
15 EST_B25 object
16 EST_M1 object
17 EST_M2 object
18 EST_M3 object
19 EST_M4 object
20 EST_MES object
21 SAT1 object
22 SAT2 object
23 EST2_1 object
24 EST2_2 object
25 EST2_3 object
26 EST2_FU object
27 EST2_TI object
28 EST2_TE object
29 EST2_MA object
30 EST2_MS object
31 EST2_T2 object
32 EST2_SA object
33 EST3 object
34 EST2_NC object
35 EST4 object
36 IDI_MT1 object
37 IDI_MT2 object
38 IDIOMAS object
39 IDIOMA1 object
40 NIV_ID1 object
41 IDIOMA2 object
42 NIV_ID2 object
43 IDIOMA3 object
44 NIV_ID3 object
45 IDIOMA4 object
46 NIV_ID4 object
47 IDIOMA5 object
48 NIV_ID5 object
49 IDIOMA6 object
50 NIV_ID6 object
51 IDIOMA7 object
52 NIV_ID7 object
53 TITIDI1 object
54 TITIDI2 object
55 TITIDI3 object
56 TIC object
57 PRAC_CR object
58 PRAC_CM object
59 E

# Inspección y ajustes

**Objetivo**: Inspección del dataset, imputación de valores perdidos debido a los itinerarios de la encuesta y creación de variables nuevas para análisis descriptivo y predictivo.   

De esta etapa del proceso se obtienen **dos datasets**:    

1- Un dataset con las **variables originales** (designadas con nombres en mayúsculas) a las que, a lo sumo, se aplica alguna de las transformaciones siguientes:  
- Imputación de valores perdidos que pueden deducirse de los itinerarios de la encuesta.  
- Sustitución de las no respuestas y las respuestas NS/NC por el valor None.  

2- Un **dataset nuevo** formado por algunas de las variables originales escogidas como candidatas a formar parte de los modelos descriptivos o predictivos y por algunas variables nuevas creadas a partir de las originales. Las variables de este dataset se designan por nombres en minúsculas para facilitar su identificación. Algunas de las transformaciones que dan lugar a este conjunto de variables son:  
- Sustitución del valor asociado a la respuesta No en las preguntas de respuesta Sí o No: se cambia el valor 2 asociado al No en el dataset original por el valor 0.  
- Creación de variables binarias para cada una de las categorías de algunas de las variables cualitativas del dataset original.  
- Creación de variables nuevas para información que puede deducirse de una o varias de las columnas del dataset original.  
- Creación de variables con información del dataset (año de la encuesta, población objeto de muestreo, etc.) con el fin de que sea posible identificar la procedencia de cada observación del el dataset que resulte de reunir todos los empleados en este estudio.  

En esta etapa **no se imputan valores a las no respuestas o a las respuestas NS/NC** que no puedan deducirse de los itinerarios de la encuesta. Solo se sustituyen por el valor None. Se procede así para evitar que los valores imputados interfieran en los resultados del análisis descriptivo. 

In [9]:
def no_respuesta(col,blank=' ',nsnc='9',tipo=None):
    """ Sustituye respuestas en blanco y NS/NC por None y establece tipo 'float' si es posible'.
    """
    if tipo=='int':
        blank=None
        nsnc=int(nsnc)
    try: df[col].replace({blank:None,nsnc:None},inplace=True)
    except: 
        serie=df[col].astype('O') # Evita conflictos cuando se aplica más de una vez
        serie.replace({blank:None,str(nsnc):None},inplace=True)
    try:df[col]=df[col].astype('float')
    except:pass # Algunas variables categóricas tienen valores no numéricos (e.g. CNAE)
    try:traduce_upper(col)
    except:pass # Para variables no categóricas (sin diccionario)
    return

In [10]:
def redefine(col,nuevo=True,cat=True,si_no=False):
    """ Crea una columna denominada 'nuevo' (por defecto, el nombre original en minúsculas) a partir 
    de la columna 'col'. 
    Si 'cat' True, asocia la variable a su diccionario y traduce la variable.
    Si si_no=True, modifica el valor asociado a respuesta No (0 en lugar de 2).  
    """
    if nuevo==True:
        nuevo=col.lower()
    if si_no==True: 
        cat=False
    df[nuevo]=df[col]
    if cat==True: 
        df[str(nuevo)+'_']=traduce_lower(nuevo)
    if si_no==True: 
        df[nuevo].replace({2:0},inplace=True)
        d_dict[nuevo]=D_dict['si_no']
        df[str(nuevo)+'_']=traduce_lower(nuevo)
    return

# BLOQUE A. DATOS PERSONALES Y SOCIODEMOGRÁFICOS

## 'IDENT' 
**Índice** único del entrevistado.

In [11]:
df['IDENT'].nunique()

30379

## Nueva variable: 'año' 
**Año** en el que se realizó la encuesta.  
Se añade para que las observaciones de este dataset sean identificables cuando se reunan varios datasets.

In [12]:
df['año']=2014 
df['año'].unique() # Comprobación

array([2014])

## Nueva variable: 'est_encuesta' 
**Estudios** oficiales objeto de la encuesta (los que definen la población objeto de muestreo).   
Se añade para que las observaciones de este dataset sean identificables cuando se reunan varios datasets.

In [13]:
df['est_encuesta']=1 
# D_dict['est_encuesta'][1]='Grado universitario'  

## 'SEXO' (redefinida como 'sexo')
**Sexo** del entrevistado.  

**'fem'**: 1= Mujer, 0=Hombre. 

In [14]:
df['SEXO'].unique() # Sin valores no informados

array([1, 6])

In [15]:
traduce_upper('SEXO')

     aux_traduce
          unique
SEXO            
1       [Hombre]
6        [Mujer]


In [16]:
# Se corrigen los códigos de la variable (homogeneidad con el resto de datasets)
df['sexo']=0
df['sexo'].mask(df['SEXO']==6,1,inplace=True)

In [17]:
df['sexo_']=traduce_lower('sexo')

     aux_traduce
          unique
sexo            
0       [Hombre]
1        [Mujer]


## 'EDAD' (redefinida como 'edad')
**Edad** del entrevistado.

In [18]:
df['EDAD'].unique() # Sin valores no informados

array([1, 2, 3])

In [19]:
traduce_upper('EDAD')

               aux_traduce
                    unique
EDAD                      
1     [Menores de 30 años]
2        [De 30 a 34 años]
3       [De 35 y más años]


In [20]:
redefine('EDAD','edad')

               aux_traduce
                    unique
edad                      
1     [Menores de 30 años]
2        [De 30 a 34 años]
3       [De 35 y más años]


In [21]:
df['edad_'].unique()

array(['Menores de 30 años', 'De 30 a 34 años', 'De 35 y más años'],
      dtype=object)

## 'NACIO'
**Nacionalidad** del entrevistado.

In [22]:
df['NACIO'].unique() # Sin valores no informados

array([1, 2])

In [23]:
traduce_upper('NACIO')

               aux_traduce
                    unique
NACIO                     
1               [Española]
2      [Otra nacionalidad]


## Nueva variable: 'nacionalidad'
**Nacionalidad española** u otra: 0=Nacionalidad no española, 1=Nacionalidad española. 

In [24]:
df['nacionalidad']=1
df['nacionalidad'].mask(df['NACIO']==2,0,inplace=True)
df[['nacionalidad','NACIO']].groupby(['NACIO']).agg(['unique']) # Comprobación

,nacionalidad
,unique
NACIO,
1,[1]
2,[0]


In [25]:
df['nacionalidad_']=traduce_lower('nacionalidad')

                             aux_traduce
                                  unique
nacionalidad                            
0             [Nacionalidad no española]
1                [Nacionalidad española]


## Nueva variable 'eu27'
**Nacido en EU27**: 1=Nacido en EU27, 0=No nacido en EU27  
  
Nota: datos insuficientes para informar la variable 'eu27' en esta encuesta. Se imputa nacimiento en eu27 a quienes declaran nacionalidad española. 

In [26]:
df['eu27']=None
df['eu27'].mask(df['nacionalidad']==1,1,inplace=True)
df[['eu27','nacionalidad']].groupby('nacionalidad').agg(['unique']) # Comprobación

,eu27
,unique
nacionalidad,
0,[None]
1,[1]


In [27]:
df['eu27_']=traduce_lower('eu27')

           aux_traduce
                unique
eu27                  
1     [Nacido en EU27]


## 'TITU' (redefinida como 'titulo_g1')
**Titulación** de grado universitario  

#### Nota: los títulos de grado, licenciatura o equivalente se han codificado con un criterio distinto al empleado en la encuesta de grado del año 2019. 

Algunas titulaciones han cambiado de denominación oficial en el tiempo transcurrido entre una encuesta y otra. Además, la encuesta de 2014 contempla distintas denominaciones (licenciatura y grado) para títulos equivalentes. El criterio de codificación empleado no tiene correspondencia con el usado en la encuesta de 2019.  

Se modifican los códigos numéricos uno a uno para corregir la discrepancia, adoptando el sistema de codificación de la encuesta de 2019. En los casos en los que la titulación antigua se corresponde con varias titulaciones en la estructura de títulos de 2019, se sustituye el título antiguo por el más frecuente entre los que son equivalentes en la oferta de grados más reciente. 

In [28]:
df['TITU'].sort_values().unique() # Sin valores no informados

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  21,  22,  24,  25,  26,  31,  32,  35,  37,
        38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,
        51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,
        64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,
        77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,
        90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102,
       103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115,
       116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128,
       129, 130, 131, 132, 998, 999])

In [29]:
# Más títulos que en encuesta de grado 2019 porque distingue licenciaturas (pre Bolonia) y grados equivalentes: 
df['TITU'].nunique()

123

In [30]:
# Traduce titulos de la encuesta 2014 con su diccionario de códigos: 
df['TITU_']=df['TITU'].astype('O').replace(D_dict['TITU'])

In [31]:
# Diccionario de correspondencias de códigos encuesta 2014 (claves) y códigos encuesta 2019 (valores):
D_actualiza_titulos={1:73101,2:81102,3:71601,4:73202,5:72401,6:82101,7:71403,8:71404,9:61901,10:71405,\
                     11:71404,12:71901,13:71101,14:73102,15:71603,16:72201,17:53203,21:41301,22:101401,24:91301,\
                     25:91501,26:92301,31:41302,32:101501,35:71401,37:73201,38:91601,39:84101,40:21301,41:31301,\
                     42:42101,43:53201,44:91101,45:91201,46:53301,47:53203,48:54101,49:51101,50:51201,51:41301,\
                     52:31101,53:41401,54:31201,55:31406,56:41403,57:23104,58:22301,59:23103,60:23103,61:23201,\
                     62:23102,63:23103,64:23201,65:23201,66:23101,67:23102,68:23201,69:23202,70:31404,71:22202,\
                     72:21302,73:31401,74:72101,75:21101,76:32102,77:104101,78:32201,79:22901,80:23901,81:104101,\
                     82:11101,83:11101,84:53101,85:41201,86:52101,87:101401,88:54201,89:21502,90:72102,91:41302,\
                     92:51202,93:22202,94:31402,95:61901,96:54201,97:32201,98:91301,99:91501,100:92301,101:41202,\
                     102:91504,103:91402,104:91505,105:41303,106:41302,107:91502,108:11901,109:104101,110:104101,\
                     111:101501,112:91503,113:11201,114:11301,115:11401,116:11401,117:11401,118:11401,119:11401,\
                     120:73102,121:71501,122:61901,123:61901,124:73202,125:71404,126:71403,127:73201,128:71601,\
                     129:81103,130:82101,131:72401,132:104101,998:109999,999:109999}

In [32]:
# Columna nueva con códigos corregidos (leyenda encuesta de grado 2019)
df['titulo_g1']=df['TITU'].replace(D_actualiza_titulos)
# Traduce de acuerdo con el diccionario de títulos de la encuesta de 2019:
df['titulo_g1_']=traduce_lower('titulo_g1') 

                                aux_traduce
                                     unique
titulo_g1                                  
11101                           [Pedagogía]
11201                  [Educación infantil]
11301                  [Educación primaria]
11401                      [Otros maestros]
11901                    [Educación social]
...                                     ...
92301                      [Trabajo social]
101401     [Actividad física y del deporte]
101501                            [Turismo]
104101      [Náutica y transporte marítimo]
109999         [Servicios (otros estudios)]

[83 rows x 1 columns]


In [33]:
# Se añade indicador de título de grado para distinguirlos de títulos de máster con idéntica denominación: 
df['titulo_g1_']='Grado '+df['titulo_g1_']

In [34]:
# Inspección de correspondencias de titulaciones: 
df[['TITU_','titulo_g1_']].groupby('TITU_').agg(['unique']).iloc[:30]

,titulo_g1_
,unique
TITU_,
Arquitecto,[Grado Arquitectura y Urbanismo y paisajismo]
Arquitecto Técnico,[Grado Arquitectura técnica]
Dip. en Biblioteconomía y Documentación,[Grado Información y documentación]
Dip. en Ciencias Empresariales,[Grado Finanzas y contabilidad]
Dip. en Educación Social,[Grado Educación social]
Dip. en Enfermería,[Grado Enfermería]
Dip. en Estadística,[Grado Estadística]
Dip. en Fisioterapia,[Grado Fisioterapia]


In [35]:
# Inspección de correspondencias de titulaciones: 
df[['TITU_','titulo_g1_']].groupby('TITU_').agg(['unique']).iloc[30:60]

,titulo_g1_
,unique
TITU_,
Ing. Geólogo,[Grado Geología]
Ing. Industrial,[Grado Ingeniería electrónica industrial y automática]
Ing. Naval y Oceánico,[Grado Ingeniería naval y oceánica]
Ing. Químico,[Grado Ingeniería química industrial e Ingeniería medioambiental]
Ing. Téc. Aeronáutico,[Grado Ingeniería aeronáutica]
Ing. Téc. Agrícola,"[Grado Ingeniería agrícola, agropecuaria y medio rural]"
Ing. Téc. Forestal,[Grado Ingeniería forestal y montes]
Ing. Téc. Industrial,[Grado Ingeniería electrónica industrial y automática]


In [36]:
# Inspección de correspondencias de titulaciones: 
df[['TITU_','titulo_g1_']].groupby('TITU_').agg(['unique']).iloc[60:90]

,titulo_g1_
,unique
TITU_,
Lic. en Biología,[Grado Biología]
Lic. en Bioquímica,[Grado Bioquímica]
Lic. en Biotecnología,[Grado Biotecnología]
Lic. en Ciencia y Tecnología de los Alimentos,[Grado Ciencia y tecnología de los alimentos e Ingeniería alimentaria]
Lic. en Ciencias Actuariales y Financieras,[Grado Financiera y actuarial]
Lic. en Ciencias Ambientales,[Grado Ciencias ambientales]
Lic. en Ciencias Políticas y de la Administración,[Grado Política y gestión pública]
Lic. en Ciencias de la Actividad Física y del Deporte,[Grado Actividad física y del deporte]


In [37]:
# Inspección de correspondencias de titulaciones: 
df[['TITU_','titulo_g1_']].groupby('TITU_').agg(['unique']).iloc[90:123]

,titulo_g1_
,unique
TITU_,
Lic. en Física,[Grado Física]
Lic. en Geografía,[Grado Geografía]
Lic. en Geología,[Grado Geología]
Lic. en Historia,[Grado Historia]
Lic. en Historia del Arte,[Grado Historia del arte]
Lic. en Historia y Ciencias de la Música,[Grado Música y Artes escénicas]
Lic. en Humanidades,[Grado Humanidades]
Lic. en Investigación y Técnicas de Mercado,[Grado Marketing]


In [38]:
df['titulo_g1'].nunique()

83

## 'RAMA' (redefinida como 'rama_g1')
**Rama de conocimiento** de la titulación.  
  
Se traduce de acuerdo con el diccionario creado durante el proceso de la encuesta de grado de 2019 para garantizar que se aplican las mismas correspondencias

In [39]:
df['RAMA'].unique() # Sin valores no informados

array([4, 3, 5, 1, 2])

In [40]:
traduce_upper('RAMA')

                          aux_traduce
                               unique
RAMA                                 
1               [Artes y humanidades]
2                          [Ciencias]
3     [Ciencias sociales y jurídicas]
4         [Ingeniería y arquitectura]
5              [Ciencias de la salud]


In [41]:
df['rama_g1']=df['titulo_g1'].replace(D_titulo_grado_a_rama)
df['rama_g1_']=traduce_lower('rama_g1')

                             aux_traduce
                                  unique
rama_g1                                 
1                  [Artes y humanidades]
2                             [Ciencias]
3        [Ciencias sociales y jurídicas]
4            [Ingeniería y arquitectura]
5                 [Ciencias de la salud]
6                            [Servicios]


In [42]:
# Inspección: 
df['rama_g1'].unique()

array([4, 2, 3, 6, 5, 1])

In [43]:
# Inspección: 
df['rama_g1_'].unique()

array(['Ingeniería y arquitectura', 'Ciencias',
       'Ciencias sociales y jurídicas', 'Servicios',
       'Ciencias de la salud', 'Artes y humanidades'], dtype=object)

In [44]:
# Inspección: 
#df[['titulo_g1_','rama_g1_']].groupby('titulo_g1_').agg(['unique'])[:30]

In [45]:
# Inspección: 
#df[['titulo_g1_','rama_g1_']].groupby('titulo_g1_').agg(['unique'])[30:60]

In [46]:
# Inspección: 
#df[['titulo_g1_','rama_g1_']].groupby('titulo_g1_').agg(['unique'])[60:90]

## Nuevas variables: 'n_art_hum', 'n_ciencias', 'n_cc_soc_ju', 'n_ing_arq', 'n_cc_salud', 'n_servicios'
**'n_art_hum'**: Número de títulos universitarios (grado, máster, doctorado o equivalente) en la rama de artes y humanidades    
**'n_ciencias'**: Número de títulos universitarios (grado, máster, doctorado o equivalente) en la rama de Ciencias  
**'n_cc_soc_ju'**: Número de títulos universitarios (grado, máster, doctorado o equivalente) en la rama de ciencias sociales y jurídicas  
**'n_ing_arq'**: Número de títulos universitarios (grado, máster, doctorado o equivalente) en la rama de ingeniería y arquitectura  
**'n_cc_salud'**: Número de títulos universitarios (grado, máster, doctorado o equivalente) en la rama de ciencias de la salud  
**'n_servicios'**: Número de títulos universitarios (grado, máster, doctorado o equivalente) en la rama de servicios  

Nota: Se crean a partir de la rama asociada al título de grado que motiva esta encuesta y se actualizan con las respuestas a preguntas posteriores. 

In [47]:
L_ramas=['n_art_hum','n_ciencias','n_cc_soc_ju','n_ing_arq','n_cc_salud']
for i,rama in enumerate (L_ramas): 
    df[rama]=0
    df[rama].mask(df['rama_g1']==i+1,1,inplace=True)
    
df[L_ramas+['rama_g1_']].groupby('rama_g1_').agg(['unique']) # Comprobación: 

,n_art_hum,n_ciencias,n_cc_soc_ju,n_ing_arq,n_cc_salud
,unique,unique,unique,unique,unique
rama_g1_,,,,,
Artes y humanidades,[1],[0],[0],[0],[0]
Ciencias,[0],[1],[0],[0],[0]
Ciencias de la salud,[0],[0],[0],[0],[1]
Ciencias sociales y jurídicas,[0],[0],[1],[0],[0]
Ingeniería y arquitectura,[0],[0],[0],[1],[0]
Servicios,[0],[0],[0],[0],[0]


In [48]:
def actualiza_ramas(col):
    """Actualiza los valores de las variables 'n_art_hum', 'n_ciencias', 'n_cc_soc_ju', 'n_ing_arq', 'n_cc_salud'
    de acuerdo con la información proporcionada por la columna 'col'.
    """
    for i,rama in enumerate(L_ramas): 
        df['incremento']=0 
        # Para cada rama, +1 si el título asociado a 'col' pertenece a la rama:
        df['incremento'].mask(df[col]==i+1,1,inplace=True) 
        df[rama]=df[rama]+df['incremento']
    return 

## 'AMBITO' (redefinida como 'ambito_g1')
**Ámbito de estudio** de titulación de grado universitario

#### Nota: Los códigos usados para etiquetar la columna 'AMBITO' no coinciden con los empleados para el ámbito de estudio de los títulos de grado en las encuesta de grado y de máster de 2019. 

Se corrige la discrepancia aplicando a la encuesta de 2014 el diccionario que mapea títulos de grado a ámbtitos de conocimiento de acuerdo con los criterios empleados en la encuesta de 2019.

In [49]:
df['ambito_g1']=df['titulo_g1'].replace(D_titulo_grado_a_ambito)
df['ambito_g1_']=traduce_lower('ambito_g1')

                                                          aux_traduce
                                                               unique
ambito_g1                                                            
11                                      [Educación (Otros estudios) ]
21                                          [Artes (Otros estudios) ]
22                                                     [Humanidades ]
23                                                         [Lenguas ]
31         [Ciencias sociales y del comportamiento (Otros estudios) ]
32                                      [Periodismo y documentación ]
41                      [Negocios y administración (Otros estudios) ]
42                                                         [Derecho ]
51                                             [Ciencias de la vida ]
52                                                  [Medio ambiente ]
53                         [Ciencias químicas, físicas y geológicas ]
54                  

In [50]:
# Inspeccción: 
df['ambito_g1'].nunique(), df['ambito_g1_'].nunique()

(32, 32)

## 'T_UNIV'
**Tipo de universidad**.

In [51]:
df['T_UNIV'].unique() # Sin valores no informados

array([1, 2])

In [52]:
traduce_upper('T_UNIV')

                  aux_traduce
                       unique
T_UNIV                       
1       [Universidad Pública]
2       [Universidad Privada]


## Nueva variable: 'tipo_centro'
**Tipo de centro educativo**: 1=Centros públicos, 0=Centros privados  

Variable proxi (en otros datasets las preguntas sobre tipo de centro están referidas a niveles educativos diferentes).

In [53]:
df['tipo_centro']=df['T_UNIV'].replace({2:0})
df['tipo_centro_']=traduce_lower('tipo_centro')
df[['T_UNIV','tipo_centro_']].groupby('tipo_centro_').agg(['unique']) # Comprobación: 

            aux_traduce
                 unique
tipo_centro            
0             [Privado]
1             [Público]


,T_UNIV
,unique
tipo_centro_,
Privado,[2]
Público,[1]


## Variable: 'tipo_docencia' 
**Formación presencial o a distancia**: 1=Formación a distancia, 0=Formación presencial    
Nota: no informada en este dataset. 

In [54]:
df['tipo_docencia']=None

## 'DISCA' (redefinida como 'discapacidad')
Tiene **discapacidad reconocida** superior al 33 %.  
Nota: Se modifica en 'disca' el valor No: 0=No. 

In [55]:
df['DISCA'].unique() # Sin valores no informados

array([0, 1])

In [56]:
traduce_upper('DISCA')

      aux_traduce
           unique
DISCA            
0            [No]
1            [Sí]


In [57]:
redefine('DISCA',nuevo='discapacidad',cat=False,si_no=True)

             aux_traduce
                  unique
discapacidad            
0                   [No]
1                   [Sí]


## 'LRES_PA' (redefinida como 'reside')
**País de residencia actual**.

In [58]:
df['LRES_PA'].sort_values().unique() # sin no informados 

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [59]:
traduce_upper('LRES_PA')

                                            aux_traduce
                                                 unique
LRES_PA                                                
1                                              [España]
2                                         [Reino Unido]
3                                            [Alemania]
4                                             [Francia]
5                       [Otro país de la Unión Europea]
6                  [Otro país europeo (fuera de la UE)]
7                        [Un país de América del Norte]
8        [Un país de América Central o América del Sur]
9                                     [Un país de Asia]
10                                  [Un país de África]
11                                 [Un país de Oceanía]


In [60]:
# Códigos para 'reside' en encuestas 2019:
[(k,v) for (k,v) in d_dict['reside'].items() if type(k)==int] # mismo criterio de codificación

[(1, 'España'),
 (2, 'Reino Unido'),
 (3, 'Alemania'),
 (9, 'Un país de Asia'),
 (4, 'Francia'),
 (5, 'Otro país de la Unión Europea'),
 (6, 'Otro país europeo (fuera de la UE)'),
 (7, 'Un país de América del Norte'),
 (8, 'Un país de América Central o América del Sur'),
 (10, 'Un país de África'),
 (11, 'Un país de Oceanía'),
 (99, 'NS/NC')]

In [61]:
redefine('LRES_PA','reside')

                                           aux_traduce
                                                unique
reside                                                
1                                             [España]
2                                        [Reino Unido]
3                                           [Alemania]
4                                            [Francia]
5                      [Otro país de la Unión Europea]
6                 [Otro país europeo (fuera de la UE)]
7                       [Un país de América del Norte]
8       [Un país de América Central o América del Sur]
9                                    [Un país de Asia]
10                                 [Un país de África]
11                                [Un país de Oceanía]


## 'ESTUDIOS_PADRE'  (redefinida como 'estudios_pa')
**Nivel de formación máximo alcanzado por el padre**.

Nota: no informada en este dataset

In [62]:
df['estudios_pa']=None

## Nueva variable: 'grado_med_pa'
**Padre** con o sin **estudios de enseñanzas medias o superiores**, incluidos los universitarios: 1=Con estudios de enseñanzas medias o superiores, 0=Sin estudios de enseñanzas medias o superiores.  

Nota: no informado en este dataset

In [63]:
df['grad_med_pa']=None

## Nueva variable: 'univ_pa'
**Padre** con o sin **estudios universitarios**: 1=Con estudios universitarios, 0=Sin estudios universitarios  

Nota: no informado en este dataset

In [64]:
df['univ_pa']=None

## 'ESTUDIOS_MADRE' (redefinida como 'estudios_ma')
**Nivel de formación máximo alcanzado por la madre**.  

Nota: no informado en este dataset

In [65]:
df['estudios_ma']=None

## Nueva variable: 'grado_med_ma'
**Madre** con o sin **estudios de enseñanzas medias o superiores**, incluidos los universitarios: 1=Con estudios de enseñanzas medias o superiores, 0=Sin estudios de enseñanzas medias o superiores.   

Nota: no informado en este dataset

In [66]:
df['grad_med_ma']=None

## Nueva variable: 'univ_ma'
**Madre** con o sin **estudios universitarios**: 1=Con estudios universitarios, 0=Sin estudios universitarios   

Nota: no informado en este dataset

In [67]:
df['univ_ma']=None

# BLOQUE B. EDUCACIÓN Y APRENDIZAJE DEL TITULADO DE MÁSTER

## 'EST_B1' (redefinida como 'beca')
¿Disfrutó de **alguna beca** durante sus estudios?: 1=Sí, 2=No, 9=NS/NC.  

Nota: se modifica en 'beca' el valor asociado a No: 0=No

In [68]:
df['EST_B1'].unique() # No informados como ' '

array(['1', '2', ' '], dtype=object)

In [69]:
traduce_upper('EST_B1')

       aux_traduce
            unique
EST_B1            
           [NS/NC]
1             [Sí]
2             [No]


In [70]:
# Se imputa None a NS/NC:
no_respuesta('EST_B1',nsnc=' ')

       aux_traduce
            unique
EST_B1            
1.0           [Sí]
2.0           [No]


In [71]:
redefine('EST_B1','beca',si_no=True)

     aux_traduce
          unique
beca            
0.0         [No]
1.0         [Sí]


## 'EST_B21' (redefinida como 'beca_general')
(Solo para respuesta Sí (1) en variable 'EST_B1' (beca))  

¿Disfrutó de una **beca general para el estudio**?: 1=Sí, 2=No, 9=NS/NC.

Nota: se modifica en 'beca_general' el valor asociado a No: 0=No

In [72]:
df['EST_B21'].unique() # No informados como '8' o ' '

array(['1', '8', '2', ' '], dtype=object)

In [73]:
traduce_upper('EST_B21')

            aux_traduce
                 unique
EST_B21                
                [NS/NC]
1                  [Sí]
2                  [No]
8        [No aplicable]


In [74]:
def re_beca(col):
    """Imputación de valores de variables condicionadas por respuesta a 'EST_B1' cuando pueden deducirse de los 
    itinerarios de la encuesta y se imputa None al el resto de valores no informados
    """
    # Se imputa respuesta No (2) a quienes manifestaron no haber disfrutado de beca (2 en 'EST_B1')
    df[col].mask(df['EST_B1']==2,2,inplace=True)
    # Se imputa None al resto de respuestas en blanco y a NS/NC:
    df[col].replace({' ':None,'8':None,},inplace=True)
    df[col]=df[col].astype('float')
    return

In [75]:
re_beca('EST_B21')

In [76]:
redefine('EST_B21','beca_general',si_no=True)

             aux_traduce
                  unique
beca_general            
0.0                 [No]
1.0                 [Sí]


In [77]:
df[['beca_general_','beca_']].groupby('beca_general_').agg(['unique']) # Comprobación: 

,beca_
,unique
beca_general_,
No,"[No, Sí]"
Sí,[Sí]


## 'EST_B22' (redefinida como 'beca_excel')
(Solo para respuesta Sí (1) en variable 'beca')  

¿Disfrutó de algún **premio o beca de excelencia**?: 1=Sí, 2=No, 9=NS/NC.  

Nota: se modifica en 'beca_excel' el valor asociado a No: 0=No.

In [78]:
df['EST_B22'].unique() # No informados como '8' o ' '

array(['2', '8', '1', ' '], dtype=object)

In [79]:
traduce_upper('EST_B22')

            aux_traduce
                 unique
EST_B22                
                [NS/NC]
1                  [Sí]
2                  [No]
8        [No aplicable]


In [80]:
re_beca('EST_B22')

In [81]:
redefine('EST_B22','beca_excel',si_no=True)

           aux_traduce
                unique
beca_excel            
0.0               [No]
1.0               [Sí]


In [82]:
df[['beca_excel_','beca_']].groupby('beca_excel_').agg(['unique']) # Comprobación: 

,beca_
,unique
beca_excel_,
No,"[Sí, No]"
Sí,[Sí]


## 'EST_B23' (redefinida como 'beca_colab')
(Solo para respuesta Sí (1) en variable 'beca')  

¿Disfrutó de alguna **beca de colaboración** en la universidad?: 1=Sí, 2=No, 9=NS/NC.  

Nota: se modifica en 'beca_colab' el valor asociado a No: 0=No.

In [83]:
df['EST_B23'].unique() # No informados como '8' o ' '

array(['2', '8', '1', ' '], dtype=object)

In [84]:
traduce_upper('EST_B23')

            aux_traduce
                 unique
EST_B23                
                [NS/NC]
1                  [Sí]
2                  [No]
8        [No aplicable]


In [85]:
re_beca('EST_B23')

In [86]:
redefine('EST_B23','beca_colab',si_no=True)

           aux_traduce
                unique
beca_colab            
0.0               [No]
1.0               [Sí]


In [87]:
df[['beca_colab_','beca_']].groupby('beca_colab_').agg(['unique']) # Comprobación: 

,beca_
,unique
beca_colab_,
No,"[Sí, No]"
Sí,[Sí]


## 'EST_B24' (redefinida como 'beca_pract')
(Solo para respuesta Sí (1) en variable 'beca')  

¿Disfrutó de alguna **beca de prácticas externas** o para prestar un servicio o desempeñar un trabajo fuera de la Universidad?: 1=Sí, 2=No, 9=NS/NC.  


Nota: se modifica en 'beca_pract' el valor asociado a No: 0=No.

In [88]:
df['EST_B24'].unique() # No informados como '8' o ' '

array(['2', '8', '1', ' '], dtype=object)

In [89]:
traduce_upper('EST_B24')

            aux_traduce
                 unique
EST_B24                
                [NS/NC]
1                  [Sí]
2                  [No]
8        [No aplicable]


In [90]:
re_beca('EST_B24')

In [91]:
redefine('EST_B24','beca_pract',si_no=True)

           aux_traduce
                unique
beca_pract            
0.0               [No]
1.0               [Sí]


In [92]:
df[['beca_pract_','beca_']].groupby('beca_pract_').agg(['unique']) # Comprobación: 

,beca_
,unique
beca_pract_,
No,"[Sí, No]"
Sí,[Sí]


## 'EST_B25' (redefinida como 'beca_extr')
(Solo para respuesta Sí (1) en variable 'beca')  

¿Disfrutó de alguna **beca para realizar estudios fuera de España** (incluidas becas complemento a programas Erasmus)?: 1=Sí, 2=No, 9=NS/NC.  

Nota: se modifica en 'beca_extr' el valor asociado a No: 0=No.

In [93]:
df['EST_B25'].unique() # No informados como '8' o ' '

array(['2', '8', '1', ' '], dtype=object)

In [94]:
traduce_upper('EST_B25')

            aux_traduce
                 unique
EST_B25                
                [NS/NC]
1                  [Sí]
2                  [No]
8        [No aplicable]


In [95]:
re_beca('EST_B25')

In [96]:
redefine('EST_B25','beca_extr',si_no=True)

          aux_traduce
               unique
beca_extr            
0.0              [No]
1.0              [Sí]


In [97]:
df[['beca_','beca_extr_']].groupby('beca_extr_').agg(['unique']) # Comprobación: 

,beca_
,unique
beca_extr_,
No,"[Sí, No]"
Sí,[Sí]


## 'EST_M1' (redefinida como 'extr')
(Solo para respuesta distinta de Sí (1) en variable 'EST_B25')  

¿Realizó parte de sus **estudios en el extranjero**?: 1=Sí, 2=No, 9=NS/NC.  

Nota: se modifica el valor asociado a No: 0=No.

In [98]:
df['EST_M1'].unique() # No informados como ' '

array(['2', '1', ' '], dtype=object)

In [99]:
traduce_upper('EST_M1')

       aux_traduce
            unique
EST_M1            
           [NS/NC]
1             [Sí]
2             [No]


In [100]:
df[['EST_B25','EST_M1']].groupby('EST_B25').agg(['unique']) # Inspección
# Imputada por el encuestador respuesta Si (1) a quienes manifestaron haber disfrutado de beca para ...
# ... estudiar fuera de España ('EST_B25')

,EST_M1
,unique
EST_B25,
1.0,[1]
2.0,"[2, 1, ]"


In [101]:
# Se imputa None a NS/NC: 
no_respuesta('EST_M1',nsnc=' ')

       aux_traduce
            unique
EST_M1            
1.0           [Sí]
2.0           [No]


In [102]:
redefine('EST_M1','extr',si_no=True)

     aux_traduce
          unique
extr            
0.0         [No]
1.0         [Sí]


In [103]:
df[['EST_M1','extr_']].groupby('extr_').agg(['unique']) # Comprobacion: 

,EST_M1
,unique
extr_,
No,[2.0]
Sí,[1.0]


## Nueva variable: 'extr_sin_beca'
¿Realizó parte de sus **estudios en el extranjero sin beca** específica?: 1=Si, 0=No.  

Variable proxi de renta familiar alta.  

In [104]:
df['extr_sin_beca']=0
df['extr_sin_beca'].mask((df['extr']==1)&(df['beca_extr']==0),1,inplace=True)

In [105]:
df['extr_sin_beca_']=traduce_lower('extr_sin_beca')
df[['extr_sin_beca_','extr_','beca_extr_']].groupby(['extr_','beca_extr_']).agg(['unique']) # Comprobación

              aux_traduce
                   unique
extr_sin_beca            
0                    [No]
1                    [Sí]


extr_sin_beca_
                         unique
extr_ beca_extr_               
No    No                   [No]
Sí    No                   [Sí]
      Sí                   [No]

## 'EST_M2' (redefinida como 'tipo_beca_extr')

(Solo para respuesta Sí (1) en variable 'EST_B2_5' (o 'beca_extr'))  

**Programa o beca de movilidad**.

In [106]:
df['EST_M2'].unique() # No informados como '8' o ' ' 

array(['8', '1', '3', ' ', '2'], dtype=object)

In [107]:
traduce_upper('EST_M2')

                                      aux_traduce
                                           unique
EST_M2                                           
                                          [NS/NC]
1                              [Programa Erasmus]
2       [Otros programas o becas dentro de la UE]
3        [Otros programas o becas fuera de la UE]
8                                  [No aplicable]


In [108]:
# Se imputa None a NS/NC (9) y a respuesta en blanco: 
no_respuesta('EST_M2',blank='8',nsnc=' ')

                                      aux_traduce
                                           unique
EST_M2                                           
1.0                            [Programa Erasmus]
2.0     [Otros programas o becas dentro de la UE]
3.0      [Otros programas o becas fuera de la UE]


In [109]:
redefine('EST_M2','tipo_beca_extr')

                                              aux_traduce
                                                   unique
tipo_beca_extr                                           
1.0                                    [Programa Erasmus]
2.0             [Otros programas o becas dentro de la UE]
3.0              [Otros programas o becas fuera de la UE]


In [110]:
df[['beca_extr_','tipo_beca_extr_']].groupby('beca_extr_',dropna=False).agg(['unique']) # Comprobación

,tipo_beca_extr_
,unique
beca_extr_,
No,[nan]
Sí,"[Programa Erasmus, Otros programas o becas fuera de la UE, Otros programas o becas dentro de la UE, nan]"
NaN,[nan]


## 'EST_M3'  (redefinida como 'pais_beca_extr')  

(Solo para respuesta Sí (1) en variable 'EST_M1' (o 'extr')) 

**País o región de la universidad de destino** cuando estudió fuera de España.

Nota: Las referencias para codificar la variable no coinciden con los empleados en las encuestas de grado y de máster de 2019. Sí coinciden las categorías de la variable. Se usan estas últimas para construir la variable nueva a partir de la literal.  

In [111]:
df['EST_M3'].sort_values().unique() # No informados como '88' o '  '

array(['  ', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10',
       '11', '12', '13', '88'], dtype=object)

In [112]:
traduce_upper('EST_M3')

                                                                 aux_traduce
                                                                      unique
EST_M3                                                                      
                                                                     [NS/NC]
01                                                                [Alemania]
02                                                                 [Francia]
03                                                                  [Italia]
04                                                                [Portugal]
05                                                             [Reino Unido]
06      [Países Nórdicos (Suecia, Noruega, Finlandia, Dinamarca e Islandia)]
07                                           [Otro país de la Unión Europea]
08                                                       [Otro país europeo]
09                                            [Un país de América del Norte]

In [113]:
no_respuesta('EST_M3',nsnc='  ', blank='88')

                                                                 aux_traduce
                                                                      unique
EST_M3                                                                      
1.0                                                               [Alemania]
2.0                                                                [Francia]
3.0                                                                 [Italia]
4.0                                                               [Portugal]
5.0                                                            [Reino Unido]
6.0     [Países Nórdicos (Suecia, Noruega, Finlandia, Dinamarca e Islandia)]
7.0                                          [Otro país de la Unión Europea]
8.0                                                      [Otro país europeo]
9.0                                           [Un país de América del Norte]
10.0                          [Un país de América Central o América del Sur]

In [114]:
# Variable 'pais_beca_extr_' literal (traducción con códigos 2014):
df['pais_beca_extr_']=df['EST_M3'].astype('O').replace(D_dict['EST_M3'])

In [115]:
# Diccionario inverso para reconstruir variable 'pais_beca_extr' numérica: 
D_inv_pais=dict([(v,k) for (k,v) in d_dict['pais_beca_extr'].items() if type(k)==int])

In [116]:
# Reconstrucción de la variable numérica a partir de la literal y el dic. inverso: 
df['pais_beca_extr']=df['pais_beca_extr_'].replace(D_inv_pais)
# Inspección (correspondencias de códigos encuestas 2014 y 2019):
df[['pais_beca_extr','EST_M3']].groupby('EST_M3').agg(['unique']) 

,pais_beca_extr
,unique
EST_M3,
1.0,[2.0]
2.0,[3.0]
3.0,[4.0]
4.0,[5.0]
5.0,[1.0]
6.0,[6.0]
7.0,[7.0]
8.0,[8.0]


## 'EST_B8' (redefinida como 'motivacion')
**Motivo principal** por el que ha realizado sus **estudios** de máster.

Nota: no informada en este dataset

In [117]:
# Se imputará el valor mediano una vez reunida esta muestra con el resto de datasets
df['motivacion']=None

## 'SAT1' (redefinida como 'sat_est_u')
¿Si tuviera que empezar de nuevo **volvería a cursar estudios universitarios**?  1=Sí, 2=No, 9=NS/NC.  

Nota: Se modifica en 'sat_est_u' el valor asociado a la respuesta No: 0=No.

In [118]:
df['SAT1'].unique() # No informados como ' '

array(['1', '2', ' '], dtype=object)

In [119]:
traduce_upper('SAT1')

     aux_traduce
          unique
SAT1            
         [NS/NC]
1           [Sí]
2           [No]


In [120]:
# Se imputa None a NS/NC:
no_respuesta('SAT1',nsnc=' ')

     aux_traduce
          unique
SAT1            
1.0         [Sí]
2.0         [No]


In [121]:
redefine('SAT1','sat_est_u',si_no=True)

          aux_traduce
               unique
sat_est_u            
0.0              [No]
1.0              [Sí]


## 'SAT2' (redefinida como 'sat_titulo')  
(Solo para respuesta Sí (1) en 'SAT1' (o 'sat_est_u')) 

¿Si tuviera que empezar de nuevo **volvería a estudiar la misma titulación**?  1=Sí, 2=No, 9=NS/NC.  

Nota: Se modifica en 'sat_titulo' el valor asociado a la respuesta No: 0=No.

In [122]:
df['SAT2'].unique() # No informados como '8' o ' '

array(['1', '2', '8', ' '], dtype=object)

In [123]:
traduce_upper('SAT2')

         aux_traduce
              unique
SAT2                
             [NS/NC]
1               [Sí]
2               [No]
8     [No aplicable]


In [124]:
# Se imputa respuesta No (2) a quienes declararon que no volverían a cursar estudios universitarios: 
df['SAT2'].mask(df['sat_est_u']==0,2,inplace=True)
# Se imputa None a resto de respuestas en blanco y NS/NC:
no_respuesta('SAT2',blank='8',nsnc=' ')

     aux_traduce
          unique
SAT2            
1.0         [Sí]
2.0         [No]


In [125]:
redefine('SAT2','sat_titulo',si_no=True)

           aux_traduce
                unique
sat_titulo            
0.0               [No]
1.0               [Sí]


## Variables sobre otra formación realizada
Nota: Debe tenerse en cuenta que se pregunta sobre otros estudios completados, no sobre los iniciados, y que para los titulados recientes se pueden estar subestimando las titulaciones posteriores por no haber habido tiempo suficiente para terminarlas. 

## 'EST_2_1'  (redefinida como 'varios_titulos_u')
¿Ha completado **otros estudios universitarios**?  1=Sí, 2=No.   

Nota: esta variable es menos específica que las incluídas en las encuestas de grado y de máster de 2019, que distinguen entre distintos niveles de formación universitaria (otros grados, máster y doctorado). 

In [126]:
df['EST2_1'].unique() # Sin no informados

array(['1', '2', ' '], dtype=object)

In [127]:
traduce_upper('EST2_1')

       aux_traduce
            unique
EST2_1            
           [NS/NC]
1             [Sí]
2             [No]


In [128]:
no_respuesta('EST2_1')

       aux_traduce
            unique
EST2_1            
1.0           [Sí]
2.0           [No]


In [129]:
redefine('EST2_1','varios_titulos_u',si_no=True)

                 aux_traduce
                      unique
varios_titulos_u            
0.0                     [No]
1.0                     [Sí]


## 'EST2_2'  (redefinida como 'est_art')
¿Ha completado otros **estudios artísticos superiores** (artes plásticas, diseño, música y danza, arte dramático o conservación y restauración de bienes culturales)?  1=Sí, 2=No.  

Nota: Se modifica en 'est_art' el valor asociado a la respuesta No: 0=No.

In [130]:
df['EST2_2'].unique() # No informados como ' '

array(['2', '1', ' '], dtype=object)

In [131]:
traduce_upper('EST2_2')

       aux_traduce
            unique
EST2_2            
           [NS/NC]
1             [Sí]
2             [No]


In [132]:
no_respuesta('EST2_2')

       aux_traduce
            unique
EST2_2            
1.0           [Sí]
2.0           [No]


In [133]:
redefine('EST2_2','est_art',si_no=True)

        aux_traduce
             unique
est_art            
0.0            [No]
1.0            [Sí]


## 'EST2_3'   (redefinida como 'g_sup')
¿Ha completado algún **ciclo formativo de grado superior** (formación profesional, artes plásticas y diseño) o **enseñanzas deportivas de grado superior**?  1=Sí, 2=No.  

Nota: Se modifica en 'g_sup' el valor asociado a la respuesta No: 0=No.

In [134]:
df['EST2_3'].unique() # No informados como ' '

array(['2', '1', ' '], dtype=object)

In [135]:
traduce_upper('EST2_3')

       aux_traduce
            unique
EST2_3            
           [NS/NC]
1             [Sí]
2             [No]


In [136]:
no_respuesta('EST2_3')

       aux_traduce
            unique
EST2_3            
1.0           [Sí]
2.0           [No]


In [137]:
redefine('EST2_3','g_sup',si_no=True)

      aux_traduce
           unique
g_sup            
0.0          [No]
1.0          [Sí]


## 'EST2_FU' (informa la variable 'extr_proxy')
¿Ha completado **otros estudios universitarios en el extranjero**?    

In [138]:
df['EST2_FU'].unique()

array(['2', '8', '1', ' '], dtype=object)

In [139]:
traduce_upper('EST2_FU')

            aux_traduce
                 unique
EST2_FU                
                [NS/NC]
1                  [Sí]
2                  [No]
8        [No aplicable]


In [140]:
# Inspección: Solo se pregunta a quienes declaran tener otros títulos universitarios
df[['EST2_FU','varios_titulos_u_']].groupby('varios_titulos_u_',dropna=False).agg(['unique'])

,EST2_FU
,unique
varios_titulos_u_,
No,[8]
Sí,"[2, 1, ]"
NaN,[ ]


## Nueva variable: 'extr_proxy'
Variable proxy de **estudios al menos en parte en el extranjero**. 

In [141]:
df['extr_proxy']=0
df['extr_proxy'].mask(df['EST2_FU']=='1',1,inplace=True)
df['extr_proxy_']=traduce_lower('extr_proxy')
df[['EST2_FU','extr_proxy_']].groupby('EST2_FU',dropna=False).agg(['unique']) # Comprobación

                                                  aux_traduce
                                                       unique
extr_proxy                                                   
0                                [No estudió fuera de España]
1           [Al menos parte de los estudios en el extranjero]


,extr_proxy_
,unique
EST2_FU,
,[No estudió fuera de España]
1,[Al menos parte de los estudios en el extranjero]
2,[No estudió fuera de España]
8,[No estudió fuera de España]


## 'EST2_TI' (informa las variables 'varios_grados', 'master', 'doctor' y 'n_grados')
Tipo de **otros estudios universitarios en el extranjero**   


In [142]:
df['EST2_TI'].unique()

array(['8', '5', ' ', '1', '6', '2', '3', '4'], dtype=object)

In [143]:
traduce_upper('EST2_TI')

                                                     aux_traduce
                                                          unique
EST2_TI                                                         
                                                         [NS/NC]
1        [Diplomatura, ingeniería técnica, arquitectura técnica]
2                      [Lincenciatura, ingeniería, arquitectura]
3                          [Grado universitario de 240 créditos]
4                   [Grado universitario de más de 240 créditos]
5                                                       [Máster]
6                                      [Doctorado universitario]
8                                                 [No aplicable]


In [144]:
# Se imputan 'varios_grados' (binaria) a quienes declaran haber estudiado otros grados, licenciaturas o ...
# ... diplomaturas en el extranjero: 
df['varios_grados']=0
df['varios_grados'].mask(df['EST2_TI'].isin(['1','2','3','4']),1,inplace=True)
df['varios_grados_']=df['varios_grados'].replace(D_dict['si_no'])
df[['varios_grados_','EST2_TI']].groupby('EST2_TI').agg(['unique']) # Comprobación: 

,varios_grados_
,unique
EST2_TI,
,[No]
1,[Sí]
2,[Sí]
3,[Sí]
4,[Sí]
5,[No]
6,[No]
8,[No]


In [145]:
# Se imputa al menos un título de grado a todos los encuestados: 
df['n_grados']=1
# Se imputan 2 a quienes declaran haber estudiado un título de grado o equivalente en el extranjero: 
df['n_grados'].mask(df['EST2_TI'].isin(['1','2','3','4']),2,inplace=True)
df[['n_grados','EST2_TI']].groupby('EST2_TI').agg(['unique']) # Comprobación: 

,n_grados
,unique
EST2_TI,
,[1]
1,[2]
2,[2]
3,[2]
4,[2]
5,[1]
6,[1]
8,[1]


In [146]:
# Se imputa 'master' (binaria) a quienes declaran haber estudiado algún título de máster en el extranjero:  
df['master']=0
df['master'].mask(df['EST2_TI']=='5',1,inplace=True)
df['master_']=df['master'].replace(D_dict['si_no'])
df[['master_','EST2_TI']].groupby('EST2_TI').agg(['unique']) # Comprobación: 

,master_
,unique
EST2_TI,
,[No]
1,[No]
2,[No]
3,[No]
4,[No]
5,[Sí]
6,[No]
8,[No]


In [147]:
# Se imputa 'doctor' (binaria) a quienes declaran haber estudiado un doctorado en el extranjero:
df['doctor']=0
df['doctor'].mask(df['EST2_TI']=='6',1,inplace=True)
df['doctor_']=df['doctor'].replace(D_dict['si_no'])
df[['doctor_','EST2_TI']].groupby('EST2_TI').agg(['unique']) # Comprobación: 

,doctor_
,unique
EST2_TI,
,[No]
1,[No]
2,[No]
3,[No]
4,[No]
5,[No]
6,[Sí]
8,[No]


## 'EST2_MA'  (informa las variables 'master', 'varios_master' y 'n_master')
¿Ha completado algún **máster en alguna universidad española**?

In [148]:
df['EST2_MA'].unique()

array(['1', '2', ' '], dtype=object)

In [149]:
traduce_upper('EST2_MA')

        aux_traduce
             unique
EST2_MA            
            [NS/NC]
1              [Sí]
2              [No]


In [150]:
# Se imputa 'master' (binaria) a quienes declaran haber estudiado uno en una universidad española: 
df['master'].mask(df['EST2_MA']=='1',1,inplace=True)
df['master_']=df['master'].replace(D_dict['si_no'])
df[['master_','EST2_MA','EST2_TI']].groupby(['EST2_MA','EST2_TI']).agg(['unique']) # Comprobación: 

master_
                 unique
EST2_MA EST2_TI        
                   [No]
        8          [No]
1                  [Sí]
        1          [Sí]
        2          [Sí]
        3          [Sí]
        4          [Sí]
        5          [Sí]
        6          [Sí]
        8          [Sí]
2                  [No]
        1          [No]
        2          [No]
        3          [No]
        4          [No]
        5          [Sí]
        6          [No]
        8          [No]

In [151]:
# Se acutualiza el valor de 'n_master' (de momento, igual a la binaria 'master'):
df['n_master']=df['master']

In [152]:
# Se imputa 'varios_master' (binaria) a quienes declaran haber estudiado uno en una universidad española y ...
# ... uno en una universidad extranjera: 
df['varios_master']=0
df['varios_master'].mask((df['EST2_MA']=='1') & (df['EST2_TI']=='5'),1,inplace=True)
df['varios_master_']=df['varios_master'].replace(D_dict['si_no'])
df[['varios_master_','EST2_MA','EST2_TI']].groupby(['EST2_MA','EST2_TI']).agg(['unique']) # Comprobación: 

varios_master_
                        unique
EST2_MA EST2_TI               
                          [No]
        8                 [No]
1                         [No]
        1                 [No]
        2                 [No]
        3                 [No]
        4                 [No]
        5                 [Sí]
        6                 [No]
        8                 [No]
2                         [No]
        1                 [No]
        2                 [No]
        3                 [No]
        4                 [No]
        5                 [No]
        6                 [No]
        8                 [No]

In [153]:
# Se actualiza el valor de 'n_master': se imputa 'n_master'=2 a quienes decaran haber estudiado uno en una ...
# ... universidad española y otro en el extranjero: 
df['n_master'].mask(df['varios_master']==1,2,inplace=True)
df[['n_master','EST2_MA','EST2_TI']].groupby(['EST2_MA','EST2_TI']).agg(['unique']) # Comprobación: 

n_master
                  unique
EST2_MA EST2_TI         
                     [0]
        8            [0]
1                    [1]
        1            [1]
        2            [1]
        3            [1]
        4            [1]
        5            [2]
        6            [1]
        8            [1]
2                    [0]
        1            [0]
        2            [0]
        3            [0]
        4            [0]
        5            [1]
        6            [0]
        8            [0]

## 'EST2_MS' (redefinida como 'rama_m1')

**Rama de conocimiento del título de máster**

In [154]:
df['EST2_MS'].unique() 

array(['5', '8', '3', '4', '1', ' ', '2'], dtype=object)

In [155]:
traduce_upper('EST2_MS')

                             aux_traduce
                                  unique
EST2_MS                                 
                                 [NS/NC]
1                             [Ciencias]
2                 [Ciencias de la salud]
3        [Ciencias sociales y jurídicas]
4                  [Artes y humanidades]
5            [Ingeniería y arquitectura]
8                         [No aplicable]


In [156]:
# Inspección: solo está informado para quienes declaran haber estudiado un título de máster en España (no en ...
# el extranjero):
df[['EST2_MA','EST2_MS']].groupby('EST2_MA').agg(['unique'])

,EST2_MS
,unique
EST2_MA,
,[ ]
1,"[5, 3, 4, 1, , 2]"
2,[8]


In [157]:
no_respuesta('EST2_MS',blank='8',nsnc=' ')

                             aux_traduce
                                  unique
EST2_MS                                 
1.0                           [Ciencias]
2.0               [Ciencias de la salud]
3.0      [Ciencias sociales y jurídicas]
4.0                [Artes y humanidades]
5.0          [Ingeniería y arquitectura]


In [158]:
redefine('EST2_MS','rama_m1')

                             aux_traduce
                                  unique
rama_m1                                 
1.0                [Artes y humanidades]
2.0                           [Ciencias]
3.0      [Ciencias sociales y jurídicas]
4.0          [Ingeniería y arquitectura]
5.0               [Ciencias de la salud]


In [159]:
# Se imputa valor 0 ('No tiene o no informado') a los valores no informados: 
df['rama_m1'].mask(df['rama_m1'].isna(),0,inplace=True)
df['rama_m1_']=traduce_lower('rama_m1')

                             aux_traduce
                                  unique
rama_m1                                 
0.0            [No tiene o no informada]
1.0                [Artes y humanidades]
2.0                           [Ciencias]
3.0      [Ciencias sociales y jurídicas]
4.0          [Ingeniería y arquitectura]
5.0               [Ciencias de la salud]


In [160]:
actualiza_ramas('rama_m1')

## 'EST2_SA'     (redefinida como 'mir')  

¿Ha realizado o está realizando alguna **formación sanitaria especializada** (MIR, EIR, FIR, PIR, BIR, RFIR, QIR, etc.)?

In [161]:
df['EST2_SA'].unique() 

array(['8', '4', ' ', '1', '2', '3', 8, 2, 4, 1], dtype=object)

In [162]:
traduce_upper('EST2_SA')

                                               aux_traduce
                                                    unique
EST2_SA                                                   
1                     [Sí, ya ha completado esa formación]
2        [Sí, esta realizando el MIR o EIR o FIR o PIR, …]
4                                                     [No]
8                 [No aplicable (Titulación no sanitaria)]
                                                   [NS/NC]
1                     [Sí, ya ha completado esa formación]
2        [Sí, esta realizando el MIR o EIR o FIR o PIR, …]
3           [No, pero está preparando el examen de acceso]
4                                                     [No]
8                 [No aplicable (Titulación no sanitaria)]


## Nueva variable: 'mir'

¿Ha **completado** alguna **formación sanitaria especializada** (MIR, EIR, FIR, PIR, BIR, RFIR, QIR, etc.)?

Nota 1: para predecir la ocupación profesional del encuestado solo es relevante la formación especializada completada.  

Nota 2: esta variable no está informada en el resto de encuestas: los sanitarios de la encuesta 2019 que tengan formación sanitaria figurarán como no 'mir' (no hay información para discriminar si la tienen o no). Por ello, es posible que la variable 'mir' introduzca más ruido que capacidad explicativa. Debe valorarse si incluirla en los modelos predictivos.

In [163]:
df['mir']=0
df['mir'].mask(df['EST2_SA'].isin([1,'1']),1,inplace=True)
df['mir_']=df['mir'].replace({0:'No',1:'Sí'})
df[['mir_','EST2_SA']].groupby('EST2_SA').agg(['unique']) # Comprobación: 

,mir_
,unique
EST2_SA,
1,[Sí]
2,[No]
4,[No]
8,[No]
,[No]
1,[Sí]
2,[No]
3,[No]


## 'EST3' (redefinida como 'g_medio')
¿Ha completado algún **ciclo formativo de grado medio** (formación profesional, artes plásticas y diseño) o **enseñanzas deportivas de grado medio**?    

In [164]:
df['EST3'].unique()

array(['2', '1', ' '], dtype=object)

In [165]:
traduce_upper('EST3')

     aux_traduce
          unique
EST3            
         [NS/NC]
1           [Sí]
2           [No]


In [166]:
no_respuesta('EST3', nsnc=' ')

     aux_traduce
          unique
EST3            
1.0         [Sí]
2.0         [No]


In [167]:
redefine('EST3','g_medio',si_no=True)

        aux_traduce
             unique
g_medio            
0.0            [No]
1.0            [Sí]


## 'IDI_MT1'   (redefinida como 'idioma_m1')

**Primer idioma materno**.

Nota: La **codificación numérica de los idiomas sigue un criterio distinto** del empleado en las encuestas de 2019. Por un lado, los códigos numéricos de una y otras encuestas no se corresponden. Por otro lado, en esta encuesta los idiomas cooficiales están agregados en una única categoría.  

Para homogeneizar la estructura, se añade a los datasets de 2019 una variable 'lengua_cooficial' que agrega la información asociada al gallego, catalán, euskera y valenciano. 

In [168]:
df['IDI_MT1'].sort_values().unique() # No informados como '  '

array(['  ', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10',
       '11'], dtype=object)

In [169]:
traduce_upper('IDI_MT1')

                                                         aux_traduce
                                                              unique
IDI_MT1                                                             
                                                             [NS/NC]
01                                                         [Español]
02       [Idioma cooficial (gallego, catalán, euskera o valenciano)]
03                                                          [Inglés]
04                                                         [Francés]
05                                                          [Alemán]
06                                                        [Italiano]
07                                                       [Portugués]
08                                                            [Ruso]
09                                                           [Árabe]
10                                                           [Chino]
11                                

In [170]:
# Códigos de codificación para idiomas en los datasets de 2019:
sorted([(k,v) for (k,v) in d_dict['idioma_m1'].items() if type(k)==int])

[(1, 'Español'),
 (2, 'Catalán'),
 (3, 'Euskera'),
 (4, 'Gallego'),
 (5, 'Valenciano'),
 (6, 'Inglés'),
 (7, 'Francés'),
 (8, 'Alemán'),
 (9, 'Italiano'),
 (10, 'Portugués'),
 (11, 'Rumano'),
 (12, 'Árabe'),
 (13, 'Chino'),
 (14, 'Ruso'),
 (15, 'Otros idiomas'),
 (16, 'lengua_cooficial'),
 (99, 'NS/NC')]

In [171]:
# Diccionario auxiliar para mapear los códigos de 2014 en los de 2019
# Categoría añadida en codificación 2019: 16 para 'lengua_cooficial'
D_actualiza_idioma={'01':1,'02':16,'03':6,'04':7,'05':8,'06':9,'07':10,'08':14,'09':12,'10':13,'11':15,'  ':None,\
                    '88':None}

In [172]:
df['idioma_m1']=df['IDI_MT1'].replace(D_actualiza_idioma)
df['idioma_m1_']=traduce_lower('idioma_m1')

                  aux_traduce
                       unique
idioma_m1                    
1.0                 [Español]
6.0                  [Inglés]
7.0                 [Francés]
8.0                  [Alemán]
9.0                [Italiano]
10.0              [Portugués]
12.0                  [Árabe]
13.0                  [Chino]
14.0                   [Ruso]
15.0          [Otros idiomas]
16.0       [lengua_cooficial]


## 'IDI_MT2'   (redefinida como 'idioma_m2')

**Segundo idioma materno**.

In [173]:
df['IDI_MT2'].sort_values().unique() # No informados como '88'  o '  ''

array(['  ', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10',
       '11', '88'], dtype=object)

In [174]:
traduce_upper('IDI_MT2')

                                                         aux_traduce
                                                              unique
IDI_MT2                                                             
                                                             [NS/NC]
01                                                         [Español]
02       [Idioma cooficial (gallego, catalán, euskera o valenciano)]
03                                                          [Inglés]
04                                                         [Francés]
05                                                          [Alemán]
06                                                        [Italiano]
07                                                       [Portugués]
08                                                            [Ruso]
09                                                           [Árabe]
10                                                           [Chino]
11                                

In [175]:
df['idioma_m2']=df['IDI_MT2'].replace(D_actualiza_idioma)
df['idioma_m2_']=traduce_lower('idioma_m1')

                  aux_traduce
                       unique
idioma_m1                    
1.0                 [Español]
6.0                  [Inglés]
7.0                 [Francés]
8.0                  [Alemán]
9.0                [Italiano]
10.0              [Portugués]
12.0                  [Árabe]
13.0                  [Chino]
14.0                   [Ruso]
15.0          [Otros idiomas]
16.0       [lengua_cooficial]


## 'IDIOMAS'   (redefinida como 'n_idiomas')

**Número de idiomas** que habla sin contar los maternos.

Nota 1: En 'n_idiomas' se computan también los maternos declarados.  
  
Nota 2: En las encuestas de 2019, 'IDIOMAS'=5 indica '5 idiomas no maternos o más'. Por homogeneidad, se **trunca** la información facilitada en este dataset con el mismo criterio. 

In [176]:
df['IDIOMAS'].unique() # No informados como ' '

array(['2', '1', '3', '4', '0', ' ', '5', '6', '7'], dtype=object)

In [177]:
#D_describe['n_idiomas']='Número de idiomas que habla incluidos los maternos'
# Se computan los idiomas maternos (al menos uno) y los no maternos declarados: 
df['aux']=0 # Hay valores no informados en 'idioma_m1'
# Al menos un idioma materno declarado: 
df['aux'].mask(df['idioma_m1'].notna() | df['idioma_m2'].notna(),1,inplace=True)
# Con dos idiomas maternos: 
df['aux'].mask(df['idioma_m1'].notna() & df['idioma_m2'].notna(),2,inplace=True)
# Comprobación:
df[['aux','idioma_m1','idioma_m2']].groupby(['idioma_m1','idioma_m2'], dropna=False).agg(['unique']) 
# Algunos encuestados no declaran ningún idioma materno (!)

aux
                    unique
idioma_m1 idioma_m2       
1.0       6.0          [2]
          7.0          [2]
          8.0          [2]
          9.0          [2]
          10.0         [2]
          12.0         [2]
          13.0         [2]
          14.0         [2]
          15.0         [2]
          16.0         [2]
          NaN          [1]
6.0       1.0          [2]
          7.0          [2]
          16.0         [2]
          NaN          [1]
7.0       1.0          [2]
          6.0          [2]
          10.0         [2]
          15.0         [2]
          16.0         [2]
          NaN          [1]
8.0       1.0          [2]
          15.0         [2]
          16.0         [2]
          NaN          [1]
9.0       1.0          [2]
          16.0         [2]
          NaN          [1]
10.0      1.0          [2]
          6.0          [2]
          16.0         [2]
          NaN          [1]
12.0      1.0          [2]
          7.0          [2]
          15.0         [2]
          NaN          [1]
13.0      15.0         [2]
          NaN          [1]
14.0      1.0          [2]
          15.0         [2]
          NaN          [1]
15.0      1.0          [2]
          6.0          [2]
          7.0          [2]
          12.0         [2]
          14.0         [2]
          16.0         [2]
          NaN          [1]
16.0      1.0          [2]
          6.0          [2]
          7.0          [2]
          8.0          [2]
          10.0         [2]
          15.0         [2]
          16.0         [2]
          NaN          [1]
NaN       NaN          [0]

In [178]:
# Se imputa None a los no informados en 'IDIOMAS' y se establece tipo 'float'
no_respuesta('IDIOMAS',nsnc=' ')
# Se imputan 0 idiomas extranjeros a los encuestados con 'IDIOMAS' no informado:
df['idiomas_extr_aux']=df['IDIOMAS'].where(df['IDIOMAS'].notna(),0)
# Se trunca el número de idiomas extranjeros a partir de 5 (homogeneidad con encuestas 2019):
df['idiomas_extr_aux'].mask(df['idiomas_extr_aux']>=5,5,inplace=True)
df[['idiomas_extr_aux','IDIOMAS']].groupby('IDIOMAS',dropna=False).agg(['unique']) # Comprobación

,idiomas_extr_aux
,unique
IDIOMAS,
0.0,[0.0]
1.0,[1.0]
2.0,[2.0]
3.0,[3.0]
4.0,[4.0]
5.0,[5.0]
6.0,[5.0]
7.0,[5.0]


In [179]:
# Se suman número de idiomas maternos y no maternos: 
df['n_idiomas']=df['aux']+df['idiomas_extr_aux'] 
del df['aux'], df['idiomas_extr_aux']
df[['IDIOMAS','n_idiomas']].groupby('IDIOMAS',dropna=False).agg(['unique']) # Comprobación

,n_idiomas
,unique
IDIOMAS,
0.0,"[1.0, 2.0]"
1.0,"[3.0, 2.0]"
2.0,"[3.0, 4.0]"
3.0,"[5.0, 4.0]"
4.0,"[5.0, 6.0]"
5.0,"[6.0, 7.0]"
6.0,"[6.0, 7.0]"
7.0,"[6.0, 7.0]"


## 'IDIOMA1'  (redefinida como 'idioma1')

**Primer idioma distinto de su lengua materna** que mejor conoce.

In [180]:
df['IDIOMA1'].sort_values().unique() # No informados como '88' o '  '

array(['  ', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10',
       '11', '88'], dtype=object)

In [181]:
traduce_upper('IDIOMA1')

                                                         aux_traduce
                                                              unique
IDIOMA1                                                             
                                                             [NS/NC]
01                                                         [Español]
02       [Idioma cooficial (gallego, catalán, euskera o valenciano)]
03                                                          [Inglés]
04                                                         [Francés]
05                                                          [Alemán]
06                                                        [Italiano]
07                                                       [Portugués]
08                                                            [Ruso]
09                                                           [Árabe]
10                                                           [Chino]
11                                

In [182]:
# Funcion re_idiomas redefinida para este dataset: 
def re_idiomas(col,tipo,nuevo=None,check=False):
    """Procesa las variables referidas a los idiomas no maternos y el nivel de conocimiento de estos. 
    """
    if tipo=='idioma': 
        nuevo=str(col).lower()
        df[nuevo]=df[col].replace(D_actualiza_idioma)
        df[str(nuevo)+'_']=df[nuevo].astype('O').replace(d_dict['idioma1'])
    if tipo=='nivel':
        no_respuesta(col,nsnc=' ',blank='8')
        df[nuevo]=df[col]
        df[str(nuevo)+'_']=df[nuevo].astype('O').replace(d_dict['idioma1_nivel'])
    # Comprobación:
    if check==True:
        df[str(col)+'_']=df[col].astype('O').replace(D_dict[col])
        print (df[[str(col)+'_',str(nuevo)+'_']].groupby(str(col)+'_').agg(['unique']))
        del df[str(col)+'_']
    return

In [183]:
re_idiomas('IDIOMA1',tipo='idioma',check=True)

                                                                     idioma1_
                                                                       unique
IDIOMA1_                                                                     
Alemán                                                               [Alemán]
Chino                                                                 [Chino]
Español                                                             [Español]
Francés                                                             [Francés]
Idioma cooficial (gallego, catalán, euskera o valenciano)  [lengua_cooficial]
Inglés                                                               [Inglés]
Italiano                                                           [Italiano]
NS/NC                                                                   [nan]
No aplicable                                                            [nan]
Otros idiomas                                                 [O

## 'NIV_ID1'  (redefinida como 'idioma1_nivel')

**Nivel de conocimiento del primer idioma no materno** que mejor conoce.

In [184]:
df['NIV_ID1'].unique() # No informados como '8' o ' '

array(['1', '2', '3', '8', ' '], dtype=object)

In [185]:
traduce_upper('NIV_ID1')

                                                                                              aux_traduce
                                                                                                   unique
NIV_ID1                                                                                                  
                                                                                                      [ ]
1        [Nivel alto (Entiende una gran variedad de textos complicados y usa el idioma con flexibilidad)]
2                    [Nivel medio (Entiende lo esencial en el lenguaje común y redacta textos sencillos)]
3                                              [Nivel bajo (Entiende y usa las expresiones más comunes )]
8                                                                                          [No aplicable]


In [186]:
re_idiomas('NIV_ID1',tipo='nivel',nuevo='idioma1_nivel',check=True)

                                                                                              aux_traduce
                                                                                                   unique
NIV_ID1                                                                                                  
1.0      [Nivel alto (Entiende una gran variedad de textos complicados y usa el idioma con flexibilidad)]
2.0                  [Nivel medio (Entiende lo esencial en el lenguaje común y redacta textos sencillos)]
3.0                                            [Nivel bajo (Entiende y usa las expresiones más comunes )]
                                                                                                                                                                                  idioma1_nivel_
                                                                                                                                                                           

## 'IDIOMA2'  (redefinida como 'idioma2')

**Segundo idioma distinto de su lengua materna** que mejor conoce.

In [187]:
df['IDIOMA2'].sort_values().unique() # No informados como '88' o '  '

array(['  ', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10',
       '11', '88'], dtype=object)

In [188]:
re_idiomas('IDIOMA2',tipo='idioma')

## 'NIV_ID2'  (redefinida como 'idioma2_nivel')

**Nivel de conocimiento del segundo idioma no materno** que mejor conoce.

In [189]:
df['NIV_ID2'].unique() # No informados como '8' o ' '

array(['2', '8', '3', '1', ' '], dtype=object)

In [190]:
re_idiomas('NIV_ID2',tipo='nivel',nuevo='idioma2_nivel')

                                                                                              aux_traduce
                                                                                                   unique
NIV_ID2                                                                                                  
1.0      [Nivel alto (Entiende una gran variedad de textos complicados y usa el idioma con flexibilidad)]
2.0                  [Nivel medio (Entiende lo esencial en el lenguaje común y redacta textos sencillos)]
3.0                                            [Nivel bajo (Entiende y usa las expresiones más comunes )]


### 'IDIOMA3'  (redefinida como 'idioma3')

**Tercer idioma distinto de su lengua materna** que mejor conoce.

In [191]:
df['IDIOMA3'].sort_values().unique() # No informados como '88' o '  '

array(['  ', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10',
       '11', '88'], dtype=object)

In [192]:
re_idiomas('IDIOMA3',tipo='idioma')

## 'NIV_ID3'  (redefinida como 'idioma3_nivel')

**Nivel de conocimiento del tercer idioma no materno** que mejor conoce.

In [193]:
df['NIV_ID3'].unique() # No informados como '8' o ' '

array(['8', '1', '3', '2', ' '], dtype=object)

In [194]:
re_idiomas('NIV_ID3',tipo='nivel',nuevo='idioma3_nivel')

                                                                                              aux_traduce
                                                                                                   unique
NIV_ID3                                                                                                  
1.0      [Nivel alto (Entiende una gran variedad de textos complicados y usa el idioma con flexibilidad)]
2.0                  [Nivel medio (Entiende lo esencial en el lenguaje común y redacta textos sencillos)]
3.0                                            [Nivel bajo (Entiende y usa las expresiones más comunes )]


## 'IDIOMA4'  (redefinida como 'idioma4')

**Cuarto idioma distinto de su lengua materna** que mejor conoce.

In [195]:
df['IDIOMA4'].sort_values().unique() # No informados como '88' o '  '

array(['  ', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10',
       '11', '88'], dtype=object)

In [196]:
re_idiomas('IDIOMA4',tipo='idioma')

## 'NIV_ID4'  (redefinida como 'idioma4_nivel')

**Nivel de conocimiento del cuarto idioma no materno** que mejor conoce.

In [197]:
df['NIV_ID4'].unique() # No informados como '8' o ' '

array(['8', '2', '3', ' ', '1'], dtype=object)

In [198]:
re_idiomas('NIV_ID4',tipo='nivel',nuevo='idioma4_nivel')

                                                                                              aux_traduce
                                                                                                   unique
NIV_ID4                                                                                                  
1.0      [Nivel alto (Entiende una gran variedad de textos complicados y usa el idioma con flexibilidad)]
2.0                  [Nivel medio (Entiende lo esencial en el lenguaje común y redacta textos sencillos)]
3.0                                            [Nivel bajo (Entiende y usa las expresiones más comunes )]


## 'IDIOMA5'  (redefinida como 'idioma5')

**Quinto idioma distinto de su lengua materna** que mejor conoce.

Nota: no informado en encuestas de 2019

In [199]:
df['IDIOMA5'].sort_values().unique() # No informados como '88' o '  '

array(['  ', '02', '04', '05', '06', '07', '08', '09', '10', '11', '88'],
      dtype=object)

In [200]:
re_idiomas('IDIOMA5',tipo='idioma')

## 'NIV_ID5'  (redefinida como 'idioma5_nivel')

**Nivel de conocimiento del quinto idioma no materno** que mejor conoce.

Nota: no informado en encuestas de 2019

In [201]:
df['NIV_ID5'].unique() # No informados como '8' o ' '

array(['8', ' ', '3', '1', '2'], dtype=object)

In [202]:
re_idiomas('NIV_ID5',tipo='nivel',nuevo='idioma5_nivel')

                                                                                              aux_traduce
                                                                                                   unique
NIV_ID5                                                                                                  
1.0      [Nivel alto (Entiende una gran variedad de textos complicados y usa el idioma con flexibilidad)]
2.0                  [Nivel medio (Entiende lo esencial en el lenguaje común y redacta textos sencillos)]
3.0                                            [Nivel bajo (Entiende y usa las expresiones más comunes )]


## 'IDIOMA6'  (redefinida como 'idioma6')

**Sexto idioma distinto de su lengua materna** que mejor conoce.

Nota: no informado en encuestas de 2019

In [203]:
df['IDIOMA6'].sort_values().unique() # No informados como '88' o '  '

array(['  ', '02', '04', '05', '06', '07', '08', '09', '10', '11', '88'],
      dtype=object)

In [204]:
re_idiomas('IDIOMA6',tipo='idioma')

## 'NIV_ID6'  (redefinida como 'idioma6_nivel')

**Nivel de conocimiento del sexto idioma no materno** que mejor conoce.

Nota: no informado en encuestas de 2019

In [205]:
df['NIV_ID6'].unique() # No informados como '8' o ' '

array(['8', ' ', '1', '3', '2'], dtype=object)

In [206]:
re_idiomas('NIV_ID6',tipo='nivel',nuevo='idioma6_nivel')

                                                                                              aux_traduce
                                                                                                   unique
NIV_ID6                                                                                                  
1.0      [Nivel alto (Entiende una gran variedad de textos complicados y usa el idioma con flexibilidad)]
2.0                  [Nivel medio (Entiende lo esencial en el lenguaje común y redacta textos sencillos)]
3.0                                            [Nivel bajo (Entiende y usa las expresiones más comunes )]


## 'IDIOMA7'  (redefinida como 'idioma7')

**Séptimo idioma distinto de su lengua materna** que mejor conoce.

Nota: no informado en encuestas de 2019

In [207]:
df['IDIOMA7'].sort_values().unique() # No informados como '88' o '  '

array(['  ', '02', '06', '08', '09', '10', '11', '88'], dtype=object)

In [208]:
re_idiomas('IDIOMA7',tipo='idioma')

## 'NIV_ID7'  (redefinida como 'idioma7_nivel')

**Nivel de conocimiento del séptimo idioma no materno** que mejor conoce.

Nota: no informado en encuestas de 2019

In [209]:
df['NIV_ID7'].unique() # No informados como '8' o ' '

array(['8', ' ', '3', '2', '1'], dtype=object)

In [210]:
re_idiomas('NIV_ID7',tipo='nivel',nuevo='idioma7_nivel')

                                                                                              aux_traduce
                                                                                                   unique
NIV_ID7                                                                                                  
1.0      [Nivel alto (Entiende una gran variedad de textos complicados y usa el idioma con flexibilidad)]
2.0                  [Nivel medio (Entiende lo esencial en el lenguaje común y redacta textos sencillos)]
3.0                                            [Nivel bajo (Entiende y usa las expresiones más comunes )]


## 'TITIDI1'  

Tiene título de la **Escuela Oficial de Idiomas**. 

In [211]:
df['TITIDI1'].unique() # No informados como ' ' 

array(['1', '2', ' '], dtype=object)

In [212]:
traduce_upper('TITIDI1')

        aux_traduce
             unique
TITIDI1            
            [NS/NC]
1              [Sí]
2              [No]


## 'TITIDI2'  

¿Tiene **título en lengua extranjera reconocido oficialmente**?

Nota: esta información aparece desagregada para cada idioma en las encuestas de 2019. La que se proporciona en esta encuesta no permite inferir en qué idioma tiene el encuestado título oficial. Por tanto, no es posible informar las variables 'idiomai_acred' en este dataset. 

In [213]:
df['TITIDI2'].unique() # No informados como ' ' 

array(['2', '1', ' '], dtype=object)

In [214]:
traduce_upper('TITIDI2')

        aux_traduce
             unique
TITIDI2            
            [NS/NC]
1              [Sí]
2              [No]


## 'TIC'  (redefinida como 'ti')

Capacidad para usar el ordenador u otros **dispositivos informáticos**.

Nota: la codificación numérica de esta variable sigue el mismo criterio que en las encuestas de 2019. No obstante, se ha modificado las denominaciones de dos niveles: 
- El nivel avanzado en 2014 se denomina intermedio en 2019 (mismas competencias).
- El nivel experto en 2014 se denomina avanzado en 2019 (mismas competencias).  

Se modifican las denominaciones de esta encuesta y de adoptan las de las encuestas de 2019.

In [215]:
df['TIC'].unique() # No informados como ' '

array(['2', '3', '1', ' '], dtype=object)

In [216]:
traduce_upper('TIC')

                                                                                                                                                                                                         aux_traduce
                                                                                                                                                                                                              unique
TIC                                                                                                                                                                                                                 
                                                                                                                                                                                                             [NS/NC]
1    [Usuario de nivel básico (navegar por internet, enviar correos electronicos, copiar o mover archivos o carpetas, escribir un texto usando un pr

In [217]:
# Diccionario para 'tic' en encuestas 2019:
[(k,v) for (k,v) in d_dict['ti'].items() if type(k)==int]

[(1, 'Usuario de nivel básico '),
 (2, 'Usuario de nivel intermedio '),
 (3, 'Usuario de nivel avanzado '),
 (9, 'NS/NC')]

In [218]:
no_respuesta('TIC',tipo='int')

                                                                                                                                                                                                         aux_traduce
                                                                                                                                                                                                              unique
TIC                                                                                                                                                                                                                 
                                                                                                                                                                                                             [NS/NC]
1    [Usuario de nivel básico (navegar por internet, enviar correos electronicos, copiar o mover archivos o carpetas, escribir un texto usando un pr

In [219]:
df['ti']=df['TIC']
df['ti_']=traduce_lower('ti')

                       aux_traduce
                            unique
ti                                
                               [ ]
1       [Usuario de nivel básico ]
2   [Usuario de nivel intermedio ]
3     [Usuario de nivel avanzado ]


# BLOQUE D. SITUACIÓN ACTUAL DEL TITULADO DE MÁSTER

## 'TRBPRN1'  (redefinida como 'sit_lab')

**Situación laboral** actual.

In [220]:
df['TRBPRN1'].unique() # Sin valores no informados

array([1, 2, 3])

In [221]:
traduce_upper('TRBPRN1')

            aux_traduce
                 unique
TRBPRN1                
1          [Trabajando]
2        [En desempleo]
3            [Inactivo]


In [222]:
# Diccionario para 'sit_lab' en encuestas 2019:
[(k,v) for (k,v) in d_dict['sit_lab'].items() if type(k)==int] # Mismo criterio de codificación

[(1, 'Trabajando'), (2, 'En desempleo'), (3, 'Inactivo')]

In [223]:
redefine('TRBPRN1','sit_lab')

            aux_traduce
                 unique
sit_lab                
1          [Trabajando]
2        [En desempleo]
3            [Inactivo]


## 'TRBPRN2'  (redefinida como 'sit_lab_detalle')

(Solo para quienes declaran **situación de inactividad** -respuesta 3 en 'TRBPRN1' (o 'sit_lab')-)

Tipo de **situación de inactividad**.  

Nota: 'sit_lab_detalle se define para todas las situaciones laborales. Se añadirá el detalle disponible para ocupados y para desempleados.

In [224]:
df['TRBPRN2'].unique() # No informados como '8' o ''

array(['8', '5', '1', ' ', '4', '2', '3'], dtype=object)

In [225]:
traduce_upper('TRBPRN2')

                                aux_traduce
                                     unique
TRBPRN2                                    
                                    [NS/NC]
1                              [Estudiante]
2                                [Jubilado]
3              [Incapacitado para trabajar]
4        [Dedicado a las labores del hogar]
5                          [Otra situación]
8                            [No aplicable]


In [226]:
# Diccionario para 'TRBPRN2' en encuestas 2019:
[(k,v) for (k,v) in d_dict['TRBPRN2'].items() if type(k)==int] # Mismo criterio de codificación

[(1, '(Inactivo) Estudiante'),
 (2, '(Inactivo) Jubilado'),
 (3, '(Inactivo) Incapacitado para trabajar'),
 (9,
  '(Ocupado) Asalariado con trabajo temporal o contrato de trabajo de duración determinada'),
 (4, '(Inactivo) Dedicado a las labores del hogar'),
 (5, '(Inactivo) Otra situación'),
 (6, 'Desempleado'),
 (7,
  '(Ocupado) Trabajador en prácticas, formación (incluido MIR, EIR, FIR,…) o becario'),
 (8,
  '(Ocupado) Asalariado con trabajo permanente o contrato de trabajo de duración indefinida'),
 (10, '(Ocupado) Empresario con asalariados'),
 (11, '(Ocupado) Trabajador independiente o empresario sin asalariados'),
 (12, '(Ocupado) Ayuda en la empresa o negocio familiar')]

In [227]:
no_respuesta('TRBPRN2',blank='8',nsnc=' ')

                                aux_traduce
                                     unique
TRBPRN2                                    
1.0                            [Estudiante]
2.0                              [Jubilado]
3.0            [Incapacitado para trabajar]
4.0      [Dedicado a las labores del hogar]
5.0                        [Otra situación]


In [228]:
df['sit_lab_detalle_']=df['TRBPRN2'].astype('O').replace(D_dict['TRBPRN2'])

In [229]:
df[['sit_lab_','sit_lab_detalle_']].groupby('sit_lab_').agg(['unique']) # Comprobación: 

,sit_lab_detalle_
,unique
sit_lab_,
En desempleo,[nan]
Inactivo,"[Otra situación, Estudiante, nan, Dedicado a las labores del hogar, Jubilado, Incapacitado para trabajar]"
Trabajando,[nan]


In [230]:
# Se añade situación laboral a la variable literal 'sit_lab_detalle_':
df['sit_lab_detalle_']='(Inactivo) '+df['sit_lab_detalle_']
df['sit_lab_detalle_'].unique()

array([nan, '(Inactivo) Otra situación', '(Inactivo) Estudiante',
       '(Inactivo) Dedicado a las labores del hogar',
       '(Inactivo) Jubilado', '(Inactivo) Incapacitado para trabajar'],
      dtype=object)

In [231]:
df['sit_lab_detalle_'][df['sit_lab']==3].value_counts(dropna=False)
# Algunos inactivos no informados en 'sit_lab_detalle_'. Se les imputa '(Inactivo) Otra situación'

(Inactivo) Estudiante                          752
NaN                                            732
(Inactivo) Otra situación                      384
(Inactivo) Dedicado a las labores del hogar     82
(Inactivo) Jubilado                             78
(Inactivo) Incapacitado para trabajar           20
Name: sit_lab_detalle_, dtype: int64

In [232]:
# Se imputa '(Inactivo) Otra situación' a NS/NC:
df['sit_lab_detalle_'].mask((df['TRBPRN2'].isna()) & (df['sit_lab']==3),'(Inactivo) Otra situación',inplace=True)

In [233]:
df['sit_lab_detalle_'][df['sit_lab']==3].value_counts(dropna=False) # Comprobación

(Inactivo) Otra situación                      1116
(Inactivo) Estudiante                           752
(Inactivo) Dedicado a las labores del hogar      82
(Inactivo) Jubilado                              78
(Inactivo) Incapacitado para trabajar            20
Name: sit_lab_detalle_, dtype: int64

## 'LTR_PAI'  (redefinida como 'pais_tr') 

**País de trabajo** actual.

In [234]:
df['LTR_PAI'].unique() # no informados como 88

array([ 3,  1, 88,  2,  9,  5, 10,  7,  4,  6, 11,  8])

In [235]:
traduce_upper('LTR_PAI')

                                            aux_traduce
                                                 unique
LTR_PAI                                                
1                                              [España]
2                                         [Reino Unido]
3                                            [Alemania]
4                                             [Francia]
5                       [Otro país de la Unión Europea]
6                  [Otro país europeo (fuera de la UE)]
7                        [Un país de América del Norte]
8        [Un país de América Central o América del Sur]
9                                     [Un país de Asia]
10                                  [Un país de África]
11                                 [Un país de Oceanía]
88                                       [No aplicable]


In [236]:
df['pais_tr']=df['LTR_PAI'].replace({88:None})
df['pais_tr'].unique()

array([3, 1, None, 2, 9, 5, 10, 7, 4, 6, 11, 8], dtype=object)

In [237]:
df['pais_tr_']=traduce_lower('pais_tr')

                                            aux_traduce
                                                 unique
pais_tr                                                
1                                              [España]
2                                         [Reino Unido]
3                                            [Alemania]
4                                             [Francia]
5                       [Otro país de la Unión Europea]
6                  [Otro país europeo (fuera de la UE)]
7                        [Un país de América del Norte]
8        [Un país de América Central o América del Sur]
9                                     [Un país de Asia]
10                                  [Un país de África]
11                                 [Un país de Oceanía]


## 'BSQTRA2'  (redefinida como 'busqueda_tr')

¿Ha **buscado trabajo** alguna vez?   

In [238]:
df['BSQTRA2'].unique() # No informados como '8'

array([8, 1, 2])

In [239]:
traduce_upper('BSQTRA2')

            aux_traduce
                 unique
BSQTRA2                
1                  [Sí]
2                  [No]
8        [No aplicable]


In [240]:
# No informada para ocupados: 
df[['sit_lab_','BSQTRA2']].groupby('sit_lab_',dropna=False).agg(['unique']) # Inspección

,BSQTRA2
,unique
sit_lab_,
En desempleo,"[8, 1]"
Inactivo,"[8, 2]"
Trabajando,[8]


In [241]:
no_respuesta('BSQTRA2',blank='8')

            aux_traduce
                 unique
BSQTRA2                
1.0                [Sí]
2.0                [No]
8.0      [No aplicable]


In [242]:
redefine('BSQTRA2','busqueda_tr',si_no=True)

            aux_traduce
                 unique
busqueda_tr            
0.0                [No]
1.0                [Sí]
8.0               [8.0]


## 'BSQCO21'  (redefinida como 'btr_internet')

(Solo para quienes declaran **situación de desempleo**)

(**Búsqueda de empleo**) ¿Busca o ha buscado empleo a través de **ofertas de trabajo en el periódico, internet**, etc.?

Nota: se informa más adelante para los ocupados cuando es posible. 

In [243]:
df['BSQCO21'].unique() # No informados como '8', 8 o ' '

array(['8', '1', '2', ' ', 8, 1, 2], dtype=object)

In [244]:
# Solo informada para desempleados: 
df[['sit_lab_','BSQCO21']].groupby('sit_lab_',dropna=False).agg(['unique']) # Inspección

,BSQCO21
,unique
sit_lab_,
En desempleo,"[8, 1, 2, , 8, 1, 2]"
Inactivo,"[8, 8]"
Trabajando,"[8, 8]"


In [245]:
def btr(col,nuevo):
    """Procesa columnas referidas a métodos de búsqueda de empleo utlizados por los desempleados.
    """
    df[col]=df[col].replace({'8':None,8:None,' ':None})
    df[col]=df[col].astype('float')
    redefine(col,nuevo,si_no=True)
    return

In [246]:
btr('BSQCO21','btr_internet')

             aux_traduce
                  unique
btr_internet            
0.0                 [No]
1.0                 [Sí]


## 'BSQCO22'  (redefinida como 'btr_spe_univ')

(Solo para quienes declaran **situación de desempleo**)

(**Búsqueda de empleo**) ¿Busca o ha buscado empleo a través de **servicios públicos de empleo o los servicios de empleo de la universidad**?  

Nota 1: se informa más adelante para los ocupados cuando es posible.  

Nota2: en esta encuesta los servicios públicos de empleo y los las universidades aparecen agregados. Se agregan también en las encuestas de 2019 por homogeniedad. 

In [247]:
df['BSQCO22'].unique() # No informados como '8', 8 o ' '

array(['8', '1', '2', ' ', 8, 1, 2], dtype=object)

In [248]:
btr('BSQCO22','btr_spe_univ')

             aux_traduce
                  unique
btr_spe_univ            
0.0                 [No]
1.0                 [Sí]


## 'BSQCO23'  (redefinida como 'btr_ett')

(Solo para quienes declaran **situación de desempleo**)

(**Búsqueda de empleo**) ¿Busca o ha buscado empleo a través de **empresas de trabajo temporal**?  

Nota: se informa más adelante para los ocupados cuando es posible.

In [249]:
df['BSQCO23'].unique() # No informados como '8', 8 o ' '

array(['8', '2', '1', ' ', 8, 2, 1], dtype=object)

In [250]:
btr('BSQCO23','btr_ett')

        aux_traduce
             unique
btr_ett            
0.0            [No]
1.0            [Sí]


## 'BSQCO24'  (redefinida como 'btr_contactos')

(Solo para quienes declaran **situación de desempleo**)

(**Búsqueda de empleo**) ¿Busca o ha buscado empleo buscando ayuda entre profesionales conocidos del sector o **contactos personales** (familia, amigos)?  

Nota: se informa más adelante para los ocupados cuando es posible.

In [251]:
df['BSQCO24'].unique() # No informados como '8', 8 o ' '

array(['8', '1', '2', ' ', 8, 2, 1], dtype=object)

In [252]:
btr('BSQCO24','btr_contactos')

              aux_traduce
                   unique
btr_contactos            
0.0                  [No]
1.0                  [Sí]


## 'BSQCO25'  (redefinida como 'btr_opo')

(Solo para quienes declaran **situación de desempleo**)

(**Búsqueda de empleo**) ¿Está preparando o preparó unas **oposiciones**?   

Nota: se informa más adelante para los ocupados cuando es posible.

In [253]:
df['BSQCO25'].unique() # No informados como '8', 8 o ' '

array(['8', '2', '1', ' ', 8, 2, 1], dtype=object)

In [254]:
btr('BSQCO25','btr_opo')

        aux_traduce
             unique
btr_opo            
0.0            [No]
1.0            [Sí]


## 'BSQCO26'  (redefinida como 'btr_auto')

(Solo para quienes declaran **situación de desempleo**)

(**Búsqueda de empleo**) ¿Busca o ha buscado equipamiento para establecer un **negocio propio**?  

Nota: se informa más adelante para los ocupados cuando es posible.

In [255]:
df['BSQCO26'].unique() # No informados como '8', 8 o ' '

array(['8', '2', '1', ' ', 8, 2, 1], dtype=object)

In [256]:
btr('BSQCO26','btr_auto')

         aux_traduce
              unique
btr_auto            
0.0             [No]
1.0             [Sí]


## 'BSQCO27'  (redefinida como 'btr_otro')

(Solo para quienes declaran **situación de desempleo**)

(**Búsqueda de empleo**) ¿Busca o ha buscado empleo de **otro modo**?  

Nota: se informa más adelante para los ocupados cuando es posible.

In [257]:
df['BSQCO27'].unique() # No informados como '8', 8 o ' '

array(['8', '2', '1', ' ', 8, 2, 1], dtype=object)

In [258]:
btr('BSQCO27','btr_otro')

         aux_traduce
              unique
btr_otro            
0.0             [No]
1.0             [Sí]


## 'BSQDIF1'  (redefinida como 'dbtr_formacion')

(Solo para quienes declaran **situación de desempleo**)

(**Dificultades para encontrar empleo**) Importancia que atribuye a las **carencias en la formación universitaria recibida** en cuanto a sus dificultades para encontrar trabajo.

In [259]:
df['BSQDIF1'].unique() # No informados como '8', 8 o ' '

array(['8', '2', '1', '3', '4', ' ', 8, 1, 2, 3, 4], dtype=object)

In [260]:
traduce_upper('BSQDIF1')

                   aux_traduce
                        unique
BSQDIF1                       
1            [Nada importante]
2            [Poco importante]
3        [Bastante importante]
4             [Muy importante]
8               [No aplicable]
                       [NS/NC]
1            [Nada importante]
2            [Poco importante]
3        [Bastante importante]
4             [Muy importante]
8               [No aplicable]


In [261]:
# Diccionario de códigos 2019:
[(k,v) for (k,v) in d_dict['dbtr_formacion'].items() if type(k)==int] # mismo criterio de codificación

[(1, 'Nada importante'),
 (2, 'Poco importante'),
 (3, 'Bastante importante'),
 (9, 'NS/NC'),
 (4, 'Muy importante')]

In [262]:
def dbtr(col,nuevo):
    """Procesa columnas referidas a las dificultades en la búsqueda de trabajo para los desempleados.
    """
    df[col]=df[col].replace({'8':None,8:None,' ':None})
    df[col]=df[col].astype('float')
    redefine(col,nuevo)
    return

In [263]:
dbtr('BSQDIF1','dbtr_formacion')

                          aux_traduce
                               unique
dbtr_formacion                       
1.0                 [Nada importante]
2.0                 [Poco importante]
3.0             [Bastante importante]
4.0                  [Muy importante]


## 'BSQDIF2'  (redefinida como 'dbtr_personales')

(Solo para quienes declaran **situación de desempleo**)

(**Dificultades para encontrar empleo**) Importancia que atribuye a las **actividades personales** que impiden trabajar (seguir estudiando, familia, otras) en cuanto a sus dificultades para encontrar trabajo.

In [264]:
df['BSQDIF2'].unique() # No informados como '8', 8 o ' '

array(['8', '4', '1', '2', '3', ' ', 8, 2, 4, 1, 3], dtype=object)

In [265]:
dbtr('BSQDIF2','dbtr_personales')

                           aux_traduce
                                unique
dbtr_personales                       
1.0                  [Nada importante]
2.0                  [Poco importante]
3.0              [Bastante importante]
4.0                   [Muy importante]


## 'BSQDIF3'  (redefinida como 'dbtr_experiencia')

(Solo para quienes declaran **situación de desempleo**)

(**Dificultades para encontrar empleo**) Importancia que atribuye a la **falta de experiencia profesional** en cuanto a sus dificultades para encontrar trabajo.

In [266]:
df['BSQDIF3'].unique() # No informados como '8' o ' '

array(['8', '4', '2', '1', '3', ' '], dtype=object)

In [267]:
dbtr('BSQDIF3','dbtr_experiencia')

                            aux_traduce
                                 unique
dbtr_experiencia                       
1.0                   [Nada importante]
2.0                   [Poco importante]
3.0               [Bastante importante]
4.0                    [Muy importante]


## 'BSQDIF4'  (redefinida como 'dbtr_expectativas')

(Solo para quienes declaran **situación de desempleo**)

(**Dificultades para encontrar empleo**) Importancia que atribuye a la **exigencia de tener un trabajo acorde con sus expectativas** económicas y profesionales en cuanto a sus dificultades para encontrar trabajo.

In [268]:
df['BSQDIF4'].unique() # No informados como '8', 8 o ' '

array(['8', '1', '2', '3', '4', ' ', 8, 4, 2, 1, 3], dtype=object)

In [269]:
dbtr('BSQDIF4','dbtr_expectativas')

                             aux_traduce
                                  unique
dbtr_expectativas                       
1.0                    [Nada importante]
2.0                    [Poco importante]
3.0                [Bastante importante]
4.0                     [Muy importante]


## 'BSQDIF5'  (redefinida como 'dbtr_idiomas')

(Solo para quienes declaran **situación de desempleo**)

(**Dificultades para encontrar empleo**) Importancia que atribuye a la **falta de conocimientos de idiomas** en cuanto a sus dificultades para encontrar trabajo.

In [270]:
df['BSQDIF5'].unique() # No informados como '8', 8 o ' '

array(['8', '2', '3', '1', '4', ' ', 8, 4, 2, 1, 3], dtype=object)

In [271]:
dbtr('BSQDIF5','dbtr_idiomas')

                        aux_traduce
                             unique
dbtr_idiomas                       
1.0               [Nada importante]
2.0               [Poco importante]
3.0           [Bastante importante]
4.0                [Muy importante]


## 'BSQDIF6'  (redefinida como 'dbtr_ti')

(Solo para quienes declaran **situación de desempleo**)

(**Dificultades para encontrar empleo**) Importancia que atribuye a la **falta de conocimientos informáticos** en cuanto a sus dificultades para encontrar trabajo.

In [272]:
df['BSQDIF6'].unique() # No informados como '8', 8 o ' '

array(['8', '2', '3', '1', ' ', '4', 8, 1, 2, 4, 3], dtype=object)

In [273]:
dbtr('BSQDIF6','dbtr_ti')

                   aux_traduce
                        unique
dbtr_ti                       
1.0          [Nada importante]
2.0          [Poco importante]
3.0      [Bastante importante]
4.0           [Muy importante]


## 'SIT_PRO' (actualiza la variable 'sit_lab_detalle')

(Solo para quienes declaran **situación de ocupación** -respuesta 1 en 'TRBPRN1' (o 'sit_lab')-)  

**Situación profesional** actual.

Nota: Los **criterios de codificación están más agregados** que los empleados en las encuestas de 2019: en este dataset no hay distinción entre empresarios con asalariados y empresarios sin asalariados o trabajadorse independientes.  


Se modifican los criterios de este dataset para hacerlos coincidir con los de 2019. Se imputa a los que declaran ser empresarios en la encuesta de 2014 la categoría 'Trabajador independiente o empresario sin asalariados', porque es más frecuente que 'Empresario con asalariados'. 

In [274]:
df['SIT_PRO'].unique() # No informados como '8' o ' ' 

array(['3', '5', '8', '4', '2', '1', ' '], dtype=object)

In [275]:
traduce_upper('SIT_PRO')

                                     aux_traduce
                                          unique
SIT_PRO                                         
                                         [NS/NC]
1            [En prácticas, formación o becario]
2           [Asalariado con contrato permanente]
3             [Asalariado con contrato temporal]
4        [Empresario o trabajador independiente]
5                               [Ayuda familiar]
8                                 [No aplicable]


In [276]:
# Diccionario para SIT_PRO' en encuestas 2019:
[(k,v) for (k,v) in d_dict['SIT_PRO'].items() if type(k)==int] 

[(1,
  'Trabajador en prácticas, formación (incluido MIR, EIR, FIR,…) o becario'),
 (2,
  'Asalariado con trabajo permanente o contrato de trabajo de duración indefinida'),
 (3,
  'Asalariado con trabajo temporal o contrato de trabajo de duración determinada'),
 (4, 'Empresario con asalariados'),
 (5, 'Trabajador independiente o empresario sin asalariados'),
 (6, 'Ayuda en la empresa o negocio familiar')]

In [277]:
no_respuesta('SIT_PRO',blank='8',nsnc=' ')

                                     aux_traduce
                                          unique
SIT_PRO                                         
1.0          [En prácticas, formación o becario]
2.0         [Asalariado con contrato permanente]
3.0           [Asalariado con contrato temporal]
4.0      [Empresario o trabajador independiente]
5.0                             [Ayuda familiar]


In [278]:
# Corrección de codificación: 
D_actualiza_sit_pro={4:5,5:6}
df['SIT_PRO'].replace(D_actualiza_sit_pro,inplace=True)
df['SIT_PRO'].unique() # Comprobación: 

array([ 3.,  6., nan,  5.,  2.,  1.])

In [279]:
# Variable literal para situación profesional de acuerdo con códigos 2019:
df['SIT_PRO_']=df['SIT_PRO'].astype('O').replace(d_dict['SIT_PRO'])

In [280]:
# Se actualiza la variable 'sit_lab_detalle_' (solo la literal para no reescribir los códigos de la variable ...
# ... discreta)
df['sit_lab_detalle_'].mask(df['sit_lab']==1,'(Ocupado) '+df['SIT_PRO_'],inplace=True)
df['sit_lab_detalle_'].mask(df['sit_lab']==2,'Desempleado',inplace=True)
df[['sit_lab_detalle_','sit_lab_']].groupby('sit_lab_',dropna=False).agg(['unique']) # Comprobación

,sit_lab_detalle_
,unique
sit_lab_,
En desempleo,[Desempleado]
Inactivo,"[(Inactivo) Otra situación, (Inactivo) Estudiante, (Inactivo) Dedicado a las labores del hogar, (Inactivo) Jubilado, (Inactivo) Incapacitado para trabajar]"
Trabajando,"[(Ocupado) Asalariado con trabajo temporal o contrato de trabajo de duración determinada, (Ocupado) Ayuda en la empresa o negocio familiar, (Ocupado) Trabajador independiente o empresario sin asalariados, (Ocupado) Asalariado con trabajo permanente o contrato de trabajo de duración indefinida, (..."


In [281]:
# Para construir la variable 'sit_lab_detalle' ('int') a partir de la literal 'sit_lab_detalle_', se emplea ...
# ... el diccionario inverso del de 'sit_lab_detalle' (este último, definido en el notebook Graduado_2019.ipynb)
[(k,v) for (k,v) in d_dict['sit_lab_detalle'].items() if type(k)==int] 

[(1, '(Inactivo) Estudiante'),
 (2, '(Inactivo) Jubilado'),
 (3, '(Inactivo) Incapacitado para trabajar'),
 (9,
  '(Ocupado) Asalariado con trabajo temporal o contrato de trabajo de duración determinada'),
 (4, '(Inactivo) Dedicado a las labores del hogar'),
 (5, '(Inactivo) Otra situación'),
 (6, 'Desempleado'),
 (7,
  '(Ocupado) Trabajador en prácticas, formación (incluido MIR, EIR, FIR,…) o becario'),
 (8,
  '(Ocupado) Asalariado con trabajo permanente o contrato de trabajo de duración indefinida'),
 (10, '(Ocupado) Empresario con asalariados'),
 (11, '(Ocupado) Trabajador independiente o empresario sin asalariados'),
 (12, '(Ocupado) Ayuda en la empresa o negocio familiar')]

In [282]:
# Diccionario inverso auxiliar para sit_lab_detalle (códigos 2019): 
D_inv_aux=dict()
for k,v in d_dict['sit_lab_detalle'].items(): 
    if type(k)==int: 
        D_inv_aux[v]=k        

In [283]:
df['sit_lab_detalle']=df['sit_lab_detalle_'].replace(D_inv_aux)
df[['sit_lab_detalle','sit_lab_detalle_','sit_lab_']].groupby('sit_lab_detalle').agg(['unique']) # Comprobación: 

,sit_lab_detalle_,sit_lab_
,unique,unique
sit_lab_detalle,,
1.0,[(Inactivo) Estudiante],[Inactivo]
2.0,[(Inactivo) Jubilado],[Inactivo]
3.0,[(Inactivo) Incapacitado para trabajar],[Inactivo]
4.0,[(Inactivo) Dedicado a las labores del hogar],[Inactivo]
5.0,[(Inactivo) Otra situación],[Inactivo]
6.0,[Desempleado],[En desempleo]
7.0,"[(Ocupado) Trabajador en prácticas, formación (incluido MIR, EIR, FIR,…) o becario]",[Trabajando]
8.0,[(Ocupado) Asalariado con trabajo permanente o contrato de trabajo de duración indefinida],[Trabajando]


## 'JORNADA'  (redefinida como 'jornada')

(Solo para quienes declaran **situación de ocupación** -respuesta 1 en 'TRBPRN1' (o 'sit_lab')-)  

Tipo de **jornada laboral** actual.

Nota: los **códigos son inversos** a los empleados en las encuestas de 2019. Se fijan los de 2019.

In [284]:
df['JORNADA'].unique() # No informados como '8' o ' ' 

array(['1', '2', '8', ' '], dtype=object)

In [285]:
traduce_upper('JORNADA')

                 aux_traduce
                      unique
JORNADA                     
                     [NS/NC]
1        [A tiempo completo]
2         [A tiempo parcial]
8            [No aplicable ]


In [286]:
# Corrección de códigos y tratamiento de no informados:
df['jornada']=df['JORNADA'].replace({'1':2,'2':1,' ':None,'8':None}) # dicc. 2019
df['jornada_']=traduce_lower('jornada')

                 aux_traduce
                      unique
jornada                     
1.0       [A tiempo parcial]
2.0      [A tiempo completo]


In [287]:
# Comprobación: 
df['JORNADA_']=df['JORNADA'].astype('O').replace(D_dict['JORNADA']) # dicc. 2014
df[['jornada_','JORNADA_']].groupby('JORNADA_').agg(['unique']) 

,jornada_
,unique
JORNADA_,
A tiempo completo,[A tiempo completo]
A tiempo parcial,[A tiempo parcial]
NS/NC,[nan]
No aplicable,[nan]


## 'TRABOC' (redefinida como 'ocupacion')

(Solo para quienes declaran **situación de ocupación** -respuesta 1 en 'TRBPRN1' (o 'sit_lab')-)  

**Ocupación** que desempeña en el puesto de trabajo actual.  
  
Nota: los **códigos numéricos** con los que se codifica la ocupación en esta encuesta **difieren** de los empleados en las de 2019. Se modifican uno a uno para establecer los criterios de codificación de 2019.

In [288]:
df['TRABOC'].sort_values().unique() # No informados como '8888'

array([ 120,  121,  133,  156,  222,  223,  237,  320,  325,  326,  328,
        329,  334,  424,  427,  437,  438,  443,  511,  512,  513,  514,
        515,  620,  621,  624,  626,  700,  724,  726,  736,  759,  831,
        832,  926,  935, 1036, 1037, 1041, 1042, 1043, 1044, 1045, 1150,
       1151, 1193, 1244, 1252, 1253, 1254, 1255, 1294, 1331, 1337, 1357,
       1358, 1483, 1484, 1558, 1581, 1591, 1592, 1594, 1673, 1674, 1675,
       1731, 1776, 1781, 1782, 1876, 1877, 1878, 1961, 1962, 1963, 1964,
       1983, 1995, 2031, 2032, 2071, 2072, 2075, 2081, 2083, 2096, 2194,
       2197, 2198, 8888])

In [289]:
df['TRABOC'].nunique()

91

In [290]:
# variable literal con códigos 2014 (útil para comprobar corrección de códigos 2014 a 2019):
df['TRABOC_']=df['TRABOC'].astype('O').replace(D_dict['TRABOC']) 

In [291]:
# Diccionario de claves de ocupación: mapea los códigos 2014 en los de 2019
D_actualiza_ocupacion={120:21,121:21,133:33,156:56,222:22,223:23,237:37,320:29,325:25,326:41,328:28,329:29,\
                       334:34,424:27,427:27,437:37,438:38,443:43,511:11,512:12,513:13,514:14,515:15,620:24,\
                       621:24,624:24,626:26,700:0,724:59,726:59,736:59,759:59,831:31,832:32,926:26,935:35,\
                       1036:36,1037:36,1041:41,1042:42,1043:43,1044:44,1045:45,1150:50,1151:51,1193:51,1244:44,\
                       1252:52,1253:53,1254:54,1255:55,1294:54,1331:31,1337:58,1357:57,1358:58,1483:83,1484:84,\
                       1558:58,1581:81,1591:91,1592:92,1594:94,1673:73,1674:74,1675:75,1731:81,1776:76,1781:81,\
                       1782:82,1876:76,1877:77,1878:78,1961:61,1962:62,1963:63,1964:64,1983:83,1995:95,2031:31,\
                       2032:32,2071:71,2072:72,2075:75,2081:81,2083:83,2096:96,2194:94,2197:97,2198:98,8888:None}

In [292]:
df['ocupacion']=df['TRABOC'].replace(D_actualiza_ocupacion) # actualiza códigos (establece los de 2019)
# variable literal auxiliar (para comprobaciones) con códigos 2019 (diccionario original, no abreviado)
df['ocupacion_aux_']=df['ocupacion'].astype('O').replace(d_dict['TRABOC'])

In [293]:
df[['ocupacion_aux_','TRABOC_']].groupby('TRABOC_').agg(['unique'])[0:30] # Comprobación: 

,ocupacion_aux_
,unique
TRABOC_,
"Agricultores, ganaderos, pescadores y sus peones; Conductor de tractor y otra maquinaria agrícola móvil","[Maquinistas de locomotoras, operadores de maquinaria agrícola y de equipos pesados móviles, y marineros: Maquinistas de locomotoras y afines; Operadores de maquinaria agrícola y forestal móvil; Operadores de otras máquinas móviles (maquinaria de movimientos de tierras, grúas, montacargas, carre..."
"Agricultores, ganaderos, pescadores y sus peones; Ganadero, pastor","[Trabajadores cualificados en actividades ganaderas, (incluidas avícolas, apícolas y similares)]"
"Agricultores, ganaderos, pescadores y sus peones; Jardinero, horticultor (huertas, viveros, invernaderos), agricultor",[Trabajadores cualificados en actividades agrícolas]
"Agricultores, ganaderos, pescadores y sus peones; Pescador, criador de especies acuáticas y trabajador forestal","[Trabajadores cualificados en actividades forestales, pesqueras y cinegéticas]"
"Agricultores, ganaderos, pescadores y sus peones; Peón agrícola, de la ganadería o la pesca","[Peones agrarios, forestales y de la pesca: Peones agrícolas; Peones ganaderos; Peones agropecuarios; Peones de la pesca, la acuicultura, forestales y de la caza]"
"Agricultores, ganaderos, pescadores y sus peones; Trabajador agropecuario",[Trabajadores cualificados en actividades agropecuarias mixtas]
"Camioneros, taxistas, mensajeros y otros conductores; Camionero, taxista, conductor de coche o furgoneta, conductor de autobús; Repartidor o mensajero en motocicleta","[Conductores de vehículos para el transporte urbano o por carretera: Conductores de automóviles, taxis y furgonetas; Conductores de autobuses y tranvías; Conductores de camiones; Conductores de motocicletas y ciclomotores]"
"Camioneros, taxistas, mensajeros y otros conductores; Conductor de trenes","[Maquinistas de locomotoras, operadores de maquinaria agrícola y de equipos pesados móviles, y marineros: Maquinistas de locomotoras y afines; Operadores de maquinaria agrícola y forestal móvil; Operadores de otras máquinas móviles (maquinaria de movimientos de tierras, grúas, montacargas, carre..."


In [294]:
df[['ocupacion_aux_','TRABOC_']].groupby('TRABOC_').agg(['unique'])[30:60] # Comprobación: 

,ocupacion_aux_
,unique
TRABOC_,
"Otras ocupaciones propias de estudios universitarios (ingeniero, arquitecto, matemático, físico, biólogo, profesional de la publicidad, grupos A1 y A2 de las AAPP …); Físico, químico, estadístico, meteorólogo, geógrafo, biólogo, zoólogo, botánico","[Profesionales de la ciencias físicas, químicas, matemáticas y de las ingenierías: Físicos, químicos, matemáticos y afines; Profesionales en ciencias naturales (biólogos, ingenieros agrónomos, de montes, técnicos agrícolas, …); Ingenieros; Arquitectos, urbanistas e ingenieros geógrafos; Ingenier..."
"Otras ocupaciones propias de estudios universitarios (ingeniero, arquitecto, matemático, físico, biólogo, profesional de la publicidad, grupos A1 y A2 de las AAPP …); Ingeniero técnico y superior (agrónomo, de montes, …)","[Profesionales de la ciencias físicas, químicas, matemáticas y de las ingenierías: Físicos, químicos, matemáticos y afines; Profesionales en ciencias naturales (biólogos, ingenieros agrónomos, de montes, técnicos agrícolas, …); Ingenieros; Arquitectos, urbanistas e ingenieros geógrafos; Ingenier..."
"Otras ocupaciones propias de estudios universitarios (ingeniero, arquitecto, matemático, físico, biólogo, profesional de la publicidad, grupos A1 y A2 de las AAPP …); Ingeniero técnico y superior (industrial, eléctrico, electrónico, de telecomunicaciones,…), arquitecto, aparejador, topógrafo, enólogo","[Profesionales de la ciencias físicas, químicas, matemáticas y de las ingenierías: Físicos, químicos, matemáticos y afines; Profesionales en ciencias naturales (biólogos, ingenieros agrónomos, de montes, técnicos agrícolas, …); Ingenieros; Arquitectos, urbanistas e ingenieros geógrafos; Ingenier..."
"Otras ocupaciones propias de estudios universitarios (ingeniero, arquitecto, matemático, físico, biólogo, profesional de la publicidad, grupos A1 y A2 de las AAPP …); Profesional en publicidad, márketing y relaciones públicas; Inspector o subinspector de Hacienda; otras ocupaciones exclusivas de las AAPP (grupos A1 o A2)",[Especialistas en organización de la Administración Pública y de las empresas y en la comercialización: Especialistas en finanzas; Especialistas en organización y administración; Técnicos de empresas y actividades turísticas; Profesionales de ventas técnicas y médicas (excepto las TIC); Otros pr...
"Peones y trabajadores manuales de industrias y servicios; Peón industrial: lavador, embotellador... a mano",[Peones de las industrias manufactureras]
"Peones y trabajadores manuales de industrias y servicios; Repartidor (publicidad, prensa gratuita, etc), ordenanza y mozo de equipaje; Lector de contadores y recaudador de máquinas expendedoras","[Recogedores de residuos urbanos, vendedores callejeros y otras ocupaciones elementales en servicios: Vendedores callejeros; Repartidores de publicidad, limpiabotas y otros trabajadores de oficios callejeros; Ordenanzas, mozos de equipaje, repartidores a pie y afines; Recogedores de residuos, cl..."
"Peones y trabajadores manuales de industrias y servicios; Reponedor de hipermercados, mozo de carga, mudanzas","[Peones del transporte, descargadores y reponedores]"
"Personal administrativo; Administrativo contable, de nóminas; auxiliar bancario; empleado administrativo de almacén, logística, transportes","[Empleados en servicios contables, financieros, y de servicios de apoyo a la producción y al transporte (Empleados de contabilidad, de control de personal y nóminas, de oficina de servicios estadísticos, financieros y bancarios, de control de abastecimientos e inventario, de logística y transpor..."


In [295]:
df[['ocupacion_aux_','TRABOC_']].groupby('TRABOC_').agg(['unique'])[60:] # Comprobación: 

,ocupacion_aux_
,unique
TRABOC_,
"Seguridad, defensa y protección civil; Guardia civil, policía nacional, autonómico, municipal, vigilante privado, guardia jurado, vigilante forestal, vigilante de prisiones, bombero","[Trabajadores de los servicios de protección y seguridad: Guardias civiles; Policías (nacionales, autonómicos y locales); Bomberos; Personal de seguridad privado; Otros trabajadores de los servicios de protección y seguridad (vigilantes de prisiones, bañistas-socorristas, agentes forestales y me..."
"Seguridad, defensa y protección civil; Inspector o comisario de Policía, oficial de la Guardia Civil","[Trabajadores de los servicios de protección y seguridad: Guardias civiles; Policías (nacionales, autonómicos y locales); Bomberos; Personal de seguridad privado; Otros trabajadores de los servicios de protección y seguridad (vigilantes de prisiones, bañistas-socorristas, agentes forestales y me..."
"Seguridad, defensa y protección civil; Oficial, suboficial, tropa o marinería de las Fuerzas Armadas",[Ocupaciones militares: Oficiales y suboficiales de las fuerzas armadas; Tropa y marinería de las fuerzas armadas]
"Seguridad, defensa y protección civil; Técnico de la Policía, suboficial de la Guardia Civil","[Trabajadores de los servicios de protección y seguridad: Guardias civiles; Policías (nacionales, autonómicos y locales); Bomberos; Personal de seguridad privado; Otros trabajadores de los servicios de protección y seguridad (vigilantes de prisiones, bañistas-socorristas, agentes forestales y me..."
"Servicio de limpieza, conserjes; Barrendero, recogedor de basura, clasificador de desechos","[Recogedores de residuos urbanos, vendedores callejeros y otras ocupaciones elementales en servicios: Vendedores callejeros; Repartidores de publicidad, limpiabotas y otros trabajadores de oficios callejeros; Ordenanzas, mozos de equipaje, repartidores a pie y afines; Recogedores de residuos, cl..."
"Servicio de limpieza, conserjes; Conserje, portero de edificios","[Trabajadores de los servicios personales: Peluqueros y especialistas en tratamientos de estética, bienestar y afines; Trabajadores que atienden a viajeros, guías turísticos y afines (auxiliares de vuelo y camareros de avión, barco y tren, revisores y cobradores de transporte terrestre, acompaña..."
"Servicio de limpieza, conserjes; Empleado doméstico",[Empleados domésticos]
"Servicio de limpieza, conserjes; Operador de máquinas de lavandería y tintorería","[Operadores de instalaciones y maquinaria fijas: Operadores en instalaciones de la extracción y explotación de minerales; Operadores en instalaciones para el tratamiento de metales; Operadores de instalaciones y máquinas de productos químicos, farmacéuticos y materiales fotosensibles; Operadores..."


In [296]:
del df['ocupacion_aux_']

In [297]:
# variable literal con códigos 2019 abreviados
df['ocupacion_']=df['ocupacion'].replace(D_ocupaciones)
# df['ocupacion_'].unique() # Comprobación: 

## 'TR_CNAE' (redefinida como 'cnae')

(Solo para quienes declaran **situación de ocupación** -respuesta 1 en 'TRBPRN1' (o 'sit_lab')-)  

**Actividad principal de la empresa** donde trabaja actualmente.   
Nota: se recodifica la variable 'cnae' para establecer valores numéricos.

In [298]:
df['TR_CNAE'].unique() # No informados como '88' o '00'

array(['M1', 'FF', '88', 'PP', 'II', '  ', 'N5', 'GG', 'LL', 'CC', 'N4',
       'OO', 'S1', 'RR', 'N6', 'N2', 'JJ', 'AA', 'EE', 'QQ', 'UU', 'S4',
       'M2', 'DD', 'N1', 'M3', 'KK', 'N3', 'HH', 'BB', 'S2', 'S3', 'TT'],
      dtype=object)

In [299]:
no_respuesta('TR_CNAE',blank='88',nsnc='  ')

                                                                                                                                                                                                   aux_traduce
                                                                                                                                                                                                        unique
TR_CNAE                                                                                                                                                                                                       
AA                                                                                                                                                              [Agricultura, ganadería, silvicultura y pesca]
BB                                                                                                                                                                          

In [300]:
df['cnae']=df['TR_CNAE'].replace(D_cnae_a_num)
df['cnae_']=traduce_lower('cnae')

                                                                                                                                                                                                aux_traduce
                                                                                                                                                                                                     unique
cnae                                                                                                                                                                                                       
0                                                                                                                                                                              [No conocido o no aplicable]
1                                                                                                                                                            [Agricultura, ganadería, si

## 'TR_TAM' (redefinida como 'tamaño_em')

(Solo para quienes declaran **situación de ocupación** -respuesta 1 en 'TRBPRN1' (o 'sit_lab')-   

**Número de trabajadores** en la empresa en la que trabaja actualmente. 

Nota: la variable está codificada con un criterio distinto del empleado en las encuestas de 2019. Se modifica el criterio para establecer el más reciente. 

In [301]:
df['TR_TAM'].unique() # No informados como '8' o ' '

array(['2', '1', '8', '3', '4', ' '], dtype=object)

In [302]:
traduce_upper('TR_TAM')

                     aux_traduce
                          unique
TR_TAM                          
                         [NS/NC]
1           [De 1 a 10 personas]
2       [Entre 11 y 19 personas]
3       [Entre 20 y 49 personas]
4            [50 o más personas]
8                 [No aplicable]


In [303]:
no_respuesta('TR_TAM',blank='8',nsnc=' ')

                     aux_traduce
                          unique
TR_TAM                          
1.0         [De 1 a 10 personas]
2.0     [Entre 11 y 19 personas]
3.0     [Entre 20 y 49 personas]
4.0          [50 o más personas]


In [304]:
# Categorías en encuestas 2019:
[(k,v) for (k,v) in d_dict['tamaño_em'].items() if type(k)==int]

[(1, 'De 1 a 9 personas'),
 (2, 'Entre 10 y 19 personas'),
 (3, 'Entre 20 y 49 personas'),
 (9, 'NS/NC'),
 (0, 'Trabajadores independientes o autónomos sin asalariados'),
 (4, 'Entre 50 y 249 personas'),
 (5, '250 o más personas')]

In [305]:
# La categoría '50 o más personas' se transforma en '250 o más personas' por ser más frecuente en las encuestas ...
# ... de 2019 que la categoría 'Entre 50 y 249 personas':
df['tamaño_em']=df['TR_TAM'].replace({4:5})
df['tamaño_em'].unique()
# Se asume el error de categorización para las empresas con 9 trabajadores

array([ 2.,  1., nan,  3.,  5.])

In [306]:
# Se imputa 'Trabajadores independientes o autónomos sin asalariados' a quienes declararon esa condición ...
# ... en 'sit_lab_detalle':
df['tamaño_em'].mask(df['SIT_PRO']==11,0,inplace=True)
df['tamaño_em_']=traduce_lower('tamaño_em') # códigos 2019
df[['tamaño_em_','sit_lab_detalle_']].groupby('tamaño_em_').agg(['unique']) # Comprobación: 

                        aux_traduce
                             unique
tamaño_em                          
1.0             [De 1 a 9 personas]
2.0        [Entre 10 y 19 personas]
3.0        [Entre 20 y 49 personas]
5.0            [250 o más personas]


,sit_lab_detalle_
,unique
tamaño_em_,
250 o más personas,"[(Ocupado) Trabajador en prácticas, formación (incluido MIR, EIR, FIR,…) o becario, (Ocupado) Asalariado con trabajo permanente o contrato de trabajo de duración indefinida, (Ocupado) Asalariado con trabajo temporal o contrato de trabajo de duración determinada, (Ocupado) Ayuda en la empresa o n..."
De 1 a 9 personas,"[(Ocupado) Ayuda en la empresa o negocio familiar, (Ocupado) Asalariado con trabajo temporal o contrato de trabajo de duración determinada, (Ocupado) Asalariado con trabajo permanente o contrato de trabajo de duración indefinida, (Ocupado) Trabajador en prácticas, formación (incluido MIR, EIR, F..."
Entre 10 y 19 personas,"[(Ocupado) Asalariado con trabajo temporal o contrato de trabajo de duración determinada, (Ocupado) Asalariado con trabajo permanente o contrato de trabajo de duración indefinida, (Ocupado) Trabajador en prácticas, formación (incluido MIR, EIR, FIR,…) o becario, (Ocupado) Ayuda en la empresa o n..."
Entre 20 y 49 personas,"[(Ocupado) Asalariado con trabajo permanente o contrato de trabajo de duración indefinida, (Ocupado) Asalariado con trabajo temporal o contrato de trabajo de duración determinada, (Ocupado) Trabajador en prácticas, formación (incluido MIR, EIR, FIR,…) o becario, (Ocupado) Ayuda en la empresa o n..."


## 'TR_SUELDO'   (redefinida como 'sueldo')  

**Sueldo mensual neto** actual.

Nota: no informada en este dataset.

In [307]:
df['sueldo']=None

# 'DSTRE41'  (redefinida como 'vbtr_teoricos')       

Influencia de los **conocimientos teóricos** para conseguir su trabajo actual. 

In [308]:
df['DSTRE41'].unique() # No informados como '8' o ' '

array(['5', '8', '4', '2', '3', '1', ' '], dtype=object)

In [309]:
traduce_upper('DSTRE41')

            aux_traduce
                 unique
DSTRE41                
                [NS/NC]
1                [Nada]
2                [Poco]
3                [Algo]
4            [Bastante]
5               [Mucho]
8        [No aplicable]


In [310]:
# Categorías en encuestas 2019:
[(k,v) for (k,v) in d_dict['vbtr_teoricos'].items() if type(k)==int] # Misma codificacion

[(1, 'Nada'),
 (2, 'Poco'),
 (3, 'Algo'),
 (9, 'NS/NC'),
 (4, 'Bastante'),
 (5, 'Mucho')]

In [311]:
def vbtr(col,nuevo):
    """Procesa columnas referidas a influencia de distintos factores en el éxito en la búsqueda de empleo.
    """
    no_respuesta(col,blank='8',nsnc=' ')
    redefine(col,nuevo)
    return

In [312]:
vbtr('DSTRE41','vbtr_teoricos')

        aux_traduce
             unique
DSTRE41            
1.0          [Nada]
2.0          [Poco]
3.0          [Algo]
4.0      [Bastante]
5.0         [Mucho]
              aux_traduce
                   unique
vbtr_teoricos            
1.0                [Nada]
2.0                [Poco]
3.0                [Algo]
4.0            [Bastante]
5.0               [Mucho]


# 'DSTRE42'  (redefinida como 'vbtr_practicos')        

Influencia de las **habilidades prácticas** para conseguir su trabajo actual. 

In [313]:
df['DSTRE42'].unique() # No informados como '8' o ' '

array(['5', '8', '4', '2', '1', '3', ' '], dtype=object)

In [314]:
vbtr('DSTRE42','vbtr_practicos')

        aux_traduce
             unique
DSTRE42            
1.0          [Nada]
2.0          [Poco]
3.0          [Algo]
4.0      [Bastante]
5.0         [Mucho]
               aux_traduce
                    unique
vbtr_practicos            
1.0                 [Nada]
2.0                 [Poco]
3.0                 [Algo]
4.0             [Bastante]
5.0                [Mucho]


# 'DSTRE43'  (redefinida como 'vbtr_idiomas')     

Influencia de **conocer idiomas** para conseguir su trabajo actual. 

In [315]:
df['DSTRE43'].unique() # No informados como '8' o ' '

array(['5', '8', '4', '2', '3', '1', ' '], dtype=object)

In [316]:
vbtr('DSTRE43','vbtr_idiomas')

        aux_traduce
             unique
DSTRE43            
1.0          [Nada]
2.0          [Poco]
3.0          [Algo]
4.0      [Bastante]
5.0         [Mucho]
             aux_traduce
                  unique
vbtr_idiomas            
1.0               [Nada]
2.0               [Poco]
3.0               [Algo]
4.0           [Bastante]
5.0              [Mucho]


# 'DSTRE44'  (redefinida como 'vbtr_ti')      

Influencia de la formación o el dominio de la **informática** y de las **tecnologías de la información y la comunicación** para conseguir su trabajo actual. 

In [317]:
df['DSTRE44'].unique() # No informados como '8' o ' '

array(['5', '8', '1', '4', '3', ' ', '2'], dtype=object)

In [318]:
vbtr('DSTRE44','vbtr_ti')

        aux_traduce
             unique
DSTRE44            
1.0          [Nada]
2.0          [Poco]
3.0          [Algo]
4.0      [Bastante]
5.0         [Mucho]
        aux_traduce
             unique
vbtr_ti            
1.0          [Nada]
2.0          [Poco]
3.0          [Algo]
4.0      [Bastante]
5.0         [Mucho]


# 'DSTRE45'  (redefinida como 'vbtr_personales')      

Influencia de las **competencias personales y sociales** -personalidad, habilidades sociales, comunicación, capacidad de trabajar en grupo- para conseguir su trabajo actual. 

In [319]:
df['DSTRE45'].unique() # No informados como '8' o ' '

array(['5', '8', '4', '3', '2', ' ', '1'], dtype=object)

In [320]:
vbtr('DSTRE45','vbtr_personales')

        aux_traduce
             unique
DSTRE45            
1.0          [Nada]
2.0          [Poco]
3.0          [Algo]
4.0      [Bastante]
5.0         [Mucho]
                aux_traduce
                     unique
vbtr_personales            
1.0                  [Nada]
2.0                  [Poco]
3.0                  [Algo]
4.0              [Bastante]
5.0                 [Mucho]


# 'DSTRE46'  (redefinida como 'vbtr_gestion')     

Influencia de la **capacidad de gestión, planificación y emprendimiento** para conseguir su trabajo actual. 

In [321]:
df['DSTRE46'].unique() # No informados como ' ' o '9'

array(['5', '8', '4', '3', '1', '2', ' '], dtype=object)

In [322]:
vbtr('DSTRE46','vbtr_gestion')

        aux_traduce
             unique
DSTRE46            
1.0          [Nada]
2.0          [Poco]
3.0          [Algo]
4.0      [Bastante]
5.0         [Mucho]
             aux_traduce
                  unique
vbtr_gestion            
1.0               [Nada]
2.0               [Poco]
3.0               [Algo]
4.0           [Bastante]
5.0              [Mucho]


# BLOQUE E. HISTORIAL LABORAL DEL GRADUADO UNIVERSITARIO

# 'HA_TRAB'  (redefinida como 'ha_trab')

Ha tenido **algún trabajo remunerado desde que terminó sus estudios** (distinto del que haya realizado mientras estudiaba), incluyendo trabajo por cuenta propia, trabajos de formación, en prácticas, puestos de becario, etc.

In [323]:
df['HA_TRAB'].unique() # Sin no informados

array([1, 2])

In [324]:
traduce_upper('HA_TRAB')

        aux_traduce
             unique
HA_TRAB            
1              [Sí]
2              [No]


In [325]:
redefine('HA_TRAB',si_no=True)

        aux_traduce
             unique
ha_trab            
0              [No]
1              [Sí]


## 'BSQCOM1'  (redefinida como 'etr_internet', actualiza 'btr_internet')

¿**Encontró trabajo** a través de ofertas de trabajo en el **periódico, internet**, etc.?  

Nota: Esta es la primera de una serie de preguntas sobre estrategias de búsqueda de empleo entre quienes han tenido alguno desde que terminaron sus estudios.   

Esta variable y otras de esta serie permiten actualizar algunas de las variables con prefijo 'btr_' sobre estrategias de búsqueda de empleo definidas originalmente solo para desempleados. 

In [326]:
df['BSQCOM1'].unique() # No informados como '8' o ' '

array(['2', ' ', '1', '8'], dtype=object)

In [327]:
traduce_upper('BSQCOM1')

            aux_traduce
                 unique
BSQCOM1                
                [NS/NC]
1                  [Sí]
2                  [No]
8        [No aplicable]


In [328]:
# Solo informada para quienes declaran haber trabajado alguna vez: 
df[['BSQCOM1','ha_trab_']].groupby('ha_trab_',dropna=False).agg(['unique'])

,BSQCOM1
,unique
ha_trab_,
No,[8]
Sí,"[2, , 1]"


In [329]:
def etr(col,nuevo):
    """Procesa las columnas referidas a las estrategias de búsqueda de empleo de quienes tuvieron alguno después
    de terminar sus estudios.
    """
    no_respuesta(col,blank='8',nsnc=' ')
    redefine(col,nuevo,si_no=True)
    return

In [330]:
etr('BSQCOM1','etr_internet')

        aux_traduce
             unique
BSQCOM1            
1.0            [Sí]
2.0            [No]
             aux_traduce
                  unique
etr_internet            
0.0                 [No]
1.0                 [Sí]


In [331]:
def actualiza_btr(col,btr):
    """ Actualiza las variables con prefijo btr_ definidas para las distintas estrategias de búsqueda de empleo, 
    añadiendo las respuestas declaradas por quienes han trabajado después de terminar sus estudios: registra la 
    estrategia definida en la variable 'col' como utilizada por el encuestado cuando este declara que le sirvió 
    para encontrar empleo. No se infiere que no la haya usado cuando el encuestado no le atribuye éxito. 
    """
    df[btr].mask(df[col]==1,1,inplace=True)
    df[str(col)+'_']=df[col].astype('O').replace(D_dict[col])
    df[str(btr)+'_']=traduce_lower(btr)
    print (df[[str(col)+'_',str(btr)+'_']][df['ha_trab']==1].groupby([str(col)+'_'],
                                                                        dropna=False).agg(['unique']))
    return

In [332]:
actualiza_btr('BSQCOM1','btr_internet')

             aux_traduce
                  unique
btr_internet            
0.0                 [No]
1.0                 [Sí]
         btr_internet_
                unique
BSQCOM1_              
No               [nan]
Sí                [Sí]
NaN              [nan]


## 'BSQCOM2'  (redefinida como 'etr_spe_univ', actualiza 'btr_spe_univ')  

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios)  

¿**Encontró trabajo** a través de los **servicios públicos de empleo o los servicios de empleo de la universidad**?

In [333]:
df['BSQCOM2'].unique() # No informados como '8' o ' '

array(['2', ' ', '1', '8'], dtype=object)

In [334]:
etr('BSQCOM2','etr_spe_univ')

        aux_traduce
             unique
BSQCOM2            
1.0            [Sí]
2.0            [No]
             aux_traduce
                  unique
etr_spe_univ            
0.0                 [No]
1.0                 [Sí]


In [335]:
actualiza_btr('BSQCOM2','btr_spe_univ')

             aux_traduce
                  unique
btr_spe_univ            
0.0                 [No]
1.0                 [Sí]
         btr_spe_univ_
                unique
BSQCOM2_              
No               [nan]
Sí                [Sí]
NaN              [nan]


## 'BSQCOM3'  (redefinida como 'etr_ett', actualiza 'btr_ett')  

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios)  

¿**Encontró trabajo** a través de **empresas de trabajo temporal**?

In [336]:
df['BSQCOM3'].unique() # No informados como '8' o ' '

array(['2', ' ', '1', '8'], dtype=object)

In [337]:
etr('BSQCOM3','etr_ett')

        aux_traduce
             unique
BSQCOM3            
1.0            [Sí]
2.0            [No]
        aux_traduce
             unique
etr_ett            
0.0            [No]
1.0            [Sí]


In [338]:
actualiza_btr('BSQCOM3','btr_ett')

        aux_traduce
             unique
btr_ett            
0.0            [No]
1.0            [Sí]
         btr_ett_
           unique
BSQCOM3_         
No          [nan]
Sí           [Sí]
NaN         [nan]


## 'BSQCOM4'  (redefinida como 'etr_contactos', actualiza 'btr_contactos')  

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios)  

¿**Encontró trabajo** contactando con el empleador por iniciativa propia o utilizando **contactos personales** (familia, amigos)

In [339]:
df['BSQCOM4'].unique() # No informados como '8' o ' '

array(['2', ' ', '1', '8'], dtype=object)

In [340]:
etr('BSQCOM4','etr_contactos')

        aux_traduce
             unique
BSQCOM4            
1.0            [Sí]
2.0            [No]
              aux_traduce
                   unique
etr_contactos            
0.0                  [No]
1.0                  [Sí]


In [341]:
actualiza_btr('BSQCOM4','btr_contactos')

              aux_traduce
                   unique
btr_contactos            
0.0                  [No]
1.0                  [Sí]
         btr_contactos_
                 unique
BSQCOM4_               
No                [nan]
Sí                 [Sí]
NaN               [nan]


## 'BSQCOM5'  (redefinida como 'etr_empleador', actualiza 'btr_otro')  

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios)  

¿**Encontró trabajo** gracias a que el **empleador se puso en contacto** con usted?

In [342]:
df['BSQCOM5'].unique() # No informados como '8' o ' '

array(['1', ' ', '2', '8'], dtype=object)

In [343]:
etr('BSQCOM5','etr_empleador')

        aux_traduce
             unique
BSQCOM5            
1.0            [Sí]
2.0            [No]
              aux_traduce
                   unique
etr_empleador            
0.0                  [No]
1.0                  [Sí]


In [344]:
actualiza_btr('BSQCOM5','btr_otro')

         aux_traduce
              unique
btr_otro            
0.0             [No]
1.0             [Sí]
         btr_otro_
            unique
BSQCOM5_          
No           [nan]
Sí            [Sí]
NaN          [nan]


## 'BSQCOM6'  (redefinida como 'etr_practicas', actualiza 'btr_otro')  

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios)  

¿Continuó con las **prácticas en empresas** o instituciones que realizó durante la carrera?

In [345]:
df['BSQCOM6'].unique() # No informados como '8' o ' '

array(['2', ' ', '1', '8'], dtype=object)

In [346]:
etr('BSQCOM6','etr_practicas')

        aux_traduce
             unique
BSQCOM6            
1.0            [Sí]
2.0            [No]
              aux_traduce
                   unique
etr_practicas            
0.0                  [No]
1.0                  [Sí]


In [347]:
actualiza_btr('BSQCOM6','btr_otro')

         aux_traduce
              unique
btr_otro            
0.0             [No]
1.0             [Sí]
          btr_otro_
             unique
BSQCOM6_           
No        [Sí, nan]
Sí             [Sí]
NaN           [nan]


## 'BSQCOM7'  (redefinida como 'etr_opo', actualiza 'btr_opo')  

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios)  

¿**Encontró trabajo** preparando unas **oposiciones**?

In [348]:
df['BSQCOM7'].unique() # No informados como '8' o ' '

array(['2', ' ', '1', '8'], dtype=object)

In [349]:
etr('BSQCOM7','etr_opo')

        aux_traduce
             unique
BSQCOM7            
1.0            [Sí]
2.0            [No]
        aux_traduce
             unique
etr_opo            
0.0            [No]
1.0            [Sí]


In [350]:
actualiza_btr('BSQCOM7','btr_opo')

        aux_traduce
             unique
btr_opo            
0.0            [No]
1.0            [Sí]
         btr_opo_
           unique
BSQCOM7_         
No          [nan]
Sí           [Sí]
NaN         [nan]


## 'BSQCOM8'  (redefinida como 'etr_auto', actualiza 'btr_auto)  

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios)  

¿**Encontró trabajo** montando un **negocio propio**?

In [351]:
df['BSQCOM8'].unique() # No informados como '8' o ' '

array(['2', ' ', '1', '8'], dtype=object)

In [352]:
etr('BSQCOM8','etr_auto')

        aux_traduce
             unique
BSQCOM8            
1.0            [Sí]
2.0            [No]
         aux_traduce
              unique
etr_auto            
0.0             [No]
1.0             [Sí]


In [353]:
actualiza_btr('BSQCOM8','btr_auto')

         aux_traduce
              unique
btr_auto            
0.0             [No]
1.0             [Sí]
         btr_auto_
            unique
BSQCOM8_          
No           [nan]
Sí            [Sí]
NaN          [nan]


## 'BSQCOM9'  (redefinida como 'etr_otro', actualiza 'btr_otro')  

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios) 

¿**Encontró trabajo de otra forma**?

In [354]:
df['BSQCOM9'].unique() # No informados como '8' o ' '

array(['2', ' ', '1', '8'], dtype=object)

In [355]:
etr('BSQCOM9','etr_otro')

        aux_traduce
             unique
BSQCOM9            
1.0            [Sí]
2.0            [No]
         aux_traduce
              unique
etr_otro            
0.0             [No]
1.0             [Sí]


In [356]:
actualiza_btr('BSQCOM9','btr_otro')

         aux_traduce
              unique
btr_otro            
0.0             [No]
1.0             [Sí]
          btr_otro_
             unique
BSQCOM9_           
No        [Sí, nan]
Sí             [Sí]
NaN           [nan]


## 'DISTRE6'  (redefinida como 'utilidad')

(Solo para quienes **declaran haber trabajado** después de haber terminado sus estudios) 

¿Cree que el título universitario le ha servido para encontrar trabajo?

In [357]:
df['DISTRE6'].unique() # No informados como '8' o ' '

array(['1', '2', ' ', '8'], dtype=object)

In [358]:
traduce_upper('DISTRE6')

            aux_traduce
                 unique
DISTRE6                
                [NS/NC]
1                  [Sí]
2                  [No]
8        [No aplicable]


In [359]:
no_respuesta('DISTRE6',blank='8',nsnc=' ')

        aux_traduce
             unique
DISTRE6            
1.0            [Sí]
2.0            [No]


In [360]:
redefine('DISTRE6','utilidad',si_no=True)

         aux_traduce
              unique
utilidad            
0.0             [No]
1.0             [Sí]


## Organización de archivos csv

In [361]:
del df['aux_traduce']

In [362]:
variables_modelos=[col for col in df.columns if col.lower()==col]
sorted(variables_modelos)

['ambito_g1',
 'ambito_g1_',
 'año',
 'beca',
 'beca_',
 'beca_colab',
 'beca_colab_',
 'beca_excel',
 'beca_excel_',
 'beca_extr',
 'beca_extr_',
 'beca_general',
 'beca_general_',
 'beca_pract',
 'beca_pract_',
 'btr_auto',
 'btr_auto_',
 'btr_contactos',
 'btr_contactos_',
 'btr_ett',
 'btr_ett_',
 'btr_internet',
 'btr_internet_',
 'btr_opo',
 'btr_opo_',
 'btr_otro',
 'btr_otro_',
 'btr_spe_univ',
 'btr_spe_univ_',
 'busqueda_tr',
 'busqueda_tr_',
 'cnae',
 'cnae_',
 'dbtr_expectativas',
 'dbtr_expectativas_',
 'dbtr_experiencia',
 'dbtr_experiencia_',
 'dbtr_formacion',
 'dbtr_formacion_',
 'dbtr_idiomas',
 'dbtr_idiomas_',
 'dbtr_personales',
 'dbtr_personales_',
 'dbtr_ti',
 'dbtr_ti_',
 'discapacidad',
 'discapacidad_',
 'doctor',
 'doctor_',
 'edad',
 'edad_',
 'est_art',
 'est_art_',
 'est_encuesta',
 'estudios_ma',
 'estudios_pa',
 'etr_auto',
 'etr_auto_',
 'etr_contactos',
 'etr_contactos_',
 'etr_empleador',
 'etr_empleador_',
 'etr_ett',
 'etr_ett_',
 'etr_internet',
 '

In [363]:
df[variables_modelos].to_csv('data/Grado_14_modelos.csv')